In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil

import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter

from tqdm import trange
from tqdm.autonotebook import tqdm


**ALL THE DATA CAN BE DOWNLOAED FROM THE LINK BELOW**

https://drive.google.com/drive/folders/1Rh7SJSfGV77UeyV3gQDwLTODvJVlSCZK?usp=sharing

**NOTE: This notebook has been run on COLAB. Please make sure it is run on COLAB to avoid any discrepancies.**

**NOTE: *Install these packages to make it compatible with the current enviornment and versions***

In [ ]:
!pip install boto3
!pip install sacremoses
!pip install sentencepiece
!pip install transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Doinng this because of getting an error while loading the dataset

# MY ERROR:Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy.
 #If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`
#!pip install transformers==3.0.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='/content/drive/MyDrive/TURL'
import os
os.chdir(path)

In [ ]:

!cp /content/drive/MyDrive/TURL /content

cp: -r not specified; omitting directory '/content/drive/MyDrive/TURL'


**Always place it in the content whilr working in COLAB. Also change made hybrid loader script regarding the directory name . Also change replace model.transformers with model.***

**Change the directory from which file is being taken.**

**Dependances:  Sentencepirece,sacremoses, transformers(been adjusted to accomodate the current version)*

**Also predefined BERT TOKENIZER doesnt work and even exist. Replace that with TOKENIZER from HuggingFace.**

In [ ]:
from data_loader.hybrid_data_loaders import *
from data_loader.header_data_loaders import *
from data_loader.CT_Wiki_data_loaders import *
from data_loader.RE_data_loaders import *
from data_loader.EL_data_loaders import *

In [ ]:


from model.configuration import TableConfig
from model.model import HybridTableMaskedLM, HybridTableCER, TableHeaderRanking, HybridTableCT,HybridTableEL,HybridTableRE,BertRE
from model.transformers import BertConfig,BertTokenizer, WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from utils.util import *
from baselines.row_population.metric import average_precision,ndcg_at_k
from baselines.cell_filling.cell_filling import *
from model import metric

In [ ]:
logger = logging.getLogger(__name__)

MODEL_CLASSES = {
    'CER': (TableConfig, HybridTableCER, BertTokenizer),
    'CF' : (TableConfig, HybridTableMaskedLM, BertTokenizer),
    'HR': (TableConfig, TableHeaderRanking, BertTokenizer),
    'CT': (TableConfig, HybridTableCT, BertTokenizer),
    'EL': (TableConfig, HybridTableEL, BertTokenizer),
    'RE': (TableConfig, HybridTableRE, BertTokenizer),
    'REBERT': (BertConfig, BertRE, BertTokenizer)
}

In [ ]:
# set data directory, this will be used to load test data
data_dir = '/content/drive/MyDrive/TURL/data'

In [ ]:
config_name = "configs/table-base-config_v2.json"
device = torch.device('cuda')
# load entity vocab from entity_vocab.txt
entity_vocab = load_entity_vocab(data_dir, ignore_bad_title=True, min_ent_count=2)
entity_wikid2id = {entity_vocab[x]['wiki_id']:x for x in entity_vocab}

total number of entity: 926135
remove because of empty title: 14206
remove because count<2: 847401


*1. entity_vocab.txt: Contains the basic statistics for each of the wikipedia tables generated by a script. Consists of count of words,wiki_id and a wiki_title.*

*eg, 986: {'count': 437,
  'mid': 'm.07bxhl',
  'wiki_id': 2421391,
  'wiki_title': 'Bhutan'}*

*2.entity_wiki2id: uses the function load_entity_vocab to extract the info from entity_vocab.txt-----wiki_id and number_of_words.*

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Table of Contents
This notebook shows examples of how to using the model components and running evaluation of different tasks.
* [Pretrained and Cell Filling](#cf)
* [Entity Linking](#el)
* [Column Type Classification](#ct)
* [Relation Extraction](#re)

<a class="anchor" id="cf"></a>
# Pretrained and CF
Here we show how to use the pretrained model to get contextualized representation for a given input table. 

We use the cell filling task for demonstration as it does not need task-specific finetuning.

**CELL FILLING TASK is not relevant to the context of our project but we do it here as the original authors of the project do it before starting with Entity Linking. Kind of a pre-processing step.**

*CF- stands for Cell Filling.*

*Only 1 step instead of 2 steps : use the pre-train to get the contextualised representation of a input table.*

**Masked Language Modelling in the context of CF.**

In [ ]:
config_class, model_class, _ = MODEL_CLASSES['CF']
config = config_class.from_pretrained(config_name)
config.output_attentions = True

# For CF, we use the base HybridTableMaskedLM, and directly load the pretrained checkpoint. The checkpoint is given by the authors and downloaded by me seperately
checkpoint = "/content/drive/MyDrive/TURL/checkpoint/output/hybrid/v2/model_v1_table_0.2_0.6_0.7_10000_1e-4_candnew_0_adam/"
model = model_class(config, is_simple=True)
#checkpoint = torch.load(os.path.join(checkpoint, 'pytorch_model.bin'))
#model.load_state_dict(checkpoint)

#--------------------REMOVE THIS------------------------------
#model.to(device)
#-------------------------------------------------------
model.eval()
# load the module for cell filling baselines
CF = cell_filling(data_dir)

**TO DO: You must restart the runtime after installing the packages/dependacies(running the cell below) while doing the first time**

In [ ]:

!pip install tf_transformers
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tf_transformers.models import BertModel
from transformers import BertTokenizer


In [ ]:
with open(os.path.join(data_dir,"CF_test_data.json"), 'r') as f:
    dev_data = json.load(f)
print('example for cell filling')
display(dev_data[1])

example for cell filling


['27295818-29',
 27295818,
 '2010–11 rangers f.c. season',
 'overall',
 '',
 ['competition', 'final position / round'],
 [[[18255941, 'UEFA Champions League'], 18255941],
  [[20795986, 'UEFA Europa League'], 20795986],
  [[28233480, 'League Cup'], 28233480],
  [[28233536, 'Scottish Cup'], 28233536]]]

In [ ]:
entity_vocab = load_entity_vocab(data_dir, ignore_bad_title=True, min_ent_count=2)

total number of entity: 926135
remove because of empty title: 14206
remove because count<2: 847401


In [ ]:

# the dataset here is the dataloader for pretraining. We use it to pass the config to construct the cell filling example
#dataset = WikiHybridTableDataset(data_dir,entity_vocab,max_cell=100, max_input_tok=350, max_input_ent=150, src="dev", max_length = [50, 10, 10], force_new=False, tokenizer = None, mode=0)
print('example of pretraining data')
with open(os.path.join(data_dir, 'dev_tables.jsonl'), 'r') as f:
    for line in f:
        example = json.loads(line.strip())
        break
display(example)

example of pretraining data


{'_id': '27289759-6',
 'column_type': [0, 0, 0, 4, 2],
 'entityCell': [[1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 0, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 0, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 0, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0],
  [1, 1, 1, 0, 0]],
 'entityColumn': [0, 1, 2],
 'entity_count': 72,
 'merged_row': [],
 'pgId': 27289759,
 'pgTitle': '2010 Santos FC season',
 'processed_tableHeaders': ['pos.', 'name', 'moving to', 'type', 'source'],
 'sectionTitle': 'Out',
 'subject_column': 1,
 'tableCaption': '',
 'tableData': [[{'surfaceLinks': [{'linkType': 'INTERNAL',
      'locType': 'MAIN_TABLE',
      'surface': 'DF',
      'target': {'id': 649702,
       'language': 'en',
       'title': 'De

**Data loader script: Change the Bert uncasse model to Hugging Face Model.**

In [ ]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """
    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    special_tokens_mask = [list(map(lambda x: 1 if x == tokenizer.pad_token_id else 0, val)) for val in labels.tolist()]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -1  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

RESERVED_ENT_VOCAB = {0:{'wiki_id':'[PAD]'},
                        1:{'wiki_id':'[ENT_MASK]'},
                        2:{'wiki_id':'[PG_ENT_MASK]'},
                        3:{'wiki_id':'[CORE_ENT_MASK]'}
                        }
RESERVED_ENT_VOCAB_NUM = len(RESERVED_ENT_VOCAB)

def process_single_hybrid_table(input_table, config):
    table_id,subject,pgEnt,pgTitle,secTitle,caption,headers,core_entities,core_entities_text,all_entities,entities,entities_text,entity_cand = input_table
    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_headers = [config.tokenizer.encode(z, max_length=config.max_header_length, add_special_tokens=False) for _,z in headers]
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    tokenized_headers_length = [len(z) for z in tokenized_headers]
    input_tok += list(itertools.chain(*tokenized_headers))
    input_tok_pos += list(itertools.chain(*[list(range(z)) for z in tokenized_headers_length]))
    input_tok_type += [1]*sum(tokenized_headers_length)
    input_ent = []
    input_ent_text = []
    input_ent_type = []
    column_en_map = {}
    row_en_map = {}
    core_entity_mask = []
    for e_i, (index, entity) in enumerate(entities):
        input_ent.append(entity)
        if len(entities_text[e_i])!=0:
            input_ent_text.append(config.tokenizer.encode(entities_text[e_i], max_length=config.max_cell_length, add_special_tokens=False))
        else:
            input_ent_text.append([])
        input_ent_type.append(3 if index[1]==subject else 4)
        core_entity_mask.append(1 if index[1]==subject else 0)
        if index[1] not in column_en_map:
            column_en_map[index[1]] = [e_i]
        else:
            column_en_map[index[1]].append(e_i)
        if index[0] not in row_en_map:
            row_en_map[index[0]] = [e_i]
        else:
            row_en_map[index[0]].append(e_i)
    input_length = len(input_tok) + len(input_ent)
    assert len(input_tok) < config.max_input_tok
    assert len(input_ent) < config.max_input_ent
    meta_and_headers_length = tokenized_meta_length+sum(tokenized_headers_length)
    assert len(input_tok) == meta_and_headers_length
    #create input mask
    tok_tok_mask = np.ones([len(input_tok), len(input_tok)], dtype=int)
    if config.src == "train":
        meta_ent_mask = np.ones([tokenized_meta_length, len(input_ent)], dtype=int)
    else:
        meta_ent_mask = np.zeros([tokenized_meta_length, len(input_ent)], dtype=int)
    header_ent_mask = np.zeros([sum(tokenized_headers_length), len(input_ent)], dtype=int)
    start_i = 0
    header_span = {}
    for h_i, (h_j, _) in enumerate(headers):
        header_span[h_j] = (start_i, start_i+tokenized_headers_length[h_i])
        start_i += tokenized_headers_length[h_i]
    for e_i, (index, _) in enumerate(entities):
        header_ent_mask[header_span[index[1]][0]:header_span[index[1]][1], e_i] = 1
    ent_header_mask = np.transpose(header_ent_mask)
    if config.src != "train":
        header_ent_mask = np.zeros([sum(tokenized_headers_length), len(input_ent)], dtype=int)

    input_tok_mask = [tok_tok_mask, np.concatenate([meta_ent_mask, header_ent_mask], axis=0)]
    ent_meta_mask = np.ones([len(input_ent), tokenized_meta_length], dtype=int)
    
    ent_ent_mask = np.eye(len(input_ent), dtype=int)
    for _,e_is in column_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                if config.src=="train" or (e_i_2<e_i_1 and input_ent[e_i_2]>=len(RESERVED_ENT_VOCAB)):
                    ent_ent_mask[e_i_1, e_i_2] = 1

    for _,e_is in row_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                if config.src=="train" or (e_i_2<e_i_1 and input_ent[e_i_2]>=len(RESERVED_ENT_VOCAB)):
                    ent_ent_mask[e_i_1, e_i_2] = 1
    input_ent_mask = [np.concatenate([ent_meta_mask, ent_header_mask], axis=1), ent_ent_mask]

    # prepend [CORE_ENT_MASK] to input, input_ent[1] = [CORE_ENT_MASK]
    input_tok_mask[1] = np.concatenate([np.zeros([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    input_ent = [config.entity_wikid2id['[CORE_ENT_MASK]']] + input_ent
    input_ent_text = [[]] + input_ent_text
    input_ent_type = [3] + input_ent_type
    # prepend pgEnt to input_ent, input_ent[0] = pgEnt
    if pgEnt!=-1:
        input_tok_mask[1] = np.concatenate([np.ones([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    else:
        input_tok_mask[1] = np.concatenate([np.zeros([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    input_ent = [pgEnt if pgEnt!=-1 else 0] + input_ent
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length]] + input_ent_text
    input_ent_type = [2] + input_ent_type

    new_input_ent_mask = [np.ones([len(input_ent), len(input_tok)], dtype=int), np.ones([len(input_ent), len(input_ent)], dtype=int)]
    new_input_ent_mask[0][2:, :] = input_ent_mask[0]
    new_input_ent_mask[1][2:, 2:] = input_ent_mask[1]
    # process [CORE_ENT_MASK] mask
    new_input_ent_mask[0][1, tokenized_meta_length:] = 0
    if subject in header_span:
        # assert header_span[0][0] == 0
        new_input_ent_mask[0][1, tokenized_meta_length+header_span[subject][0]:tokenized_meta_length+header_span[subject][1]] = 1
    new_input_ent_mask[1][1, 2:] = 0
    new_input_ent_mask[1][2:, 1] = 0
    if subject in column_en_map:
        new_input_ent_mask[1][1, 2+column_en_map[subject][0]] = 1 # seed=1
    # process pgEnt mask
    if pgEnt==-1:
        new_input_ent_mask[1][:, 0] = 0
        new_input_ent_mask[1][0, :] = 0
    input_ent_mask = new_input_ent_mask
    core_entity_mask = [0,1]+core_entity_mask
    
    if entity_cand is not None:
        entity_cand = list(set(entity_cand)-all_entities)
    all_entity_set = list(all_entities)

    def find_id(e, e_list):
        for i, e_1 in enumerate(e_list):
            if e == e_1:
                return i
        # pdb.set_trace()
        raise Exception
    if config.src == "train":
        input_ent_local_id = [find_id(pgEnt, all_entity_set) if pgEnt!=-1 else 0,0]+[find_id(e, all_entity_set) for e in input_ent[2:]]
        exclusive_ent_mask = np.full([len(input_ent), max([len(z) for _,z in column_en_map.items()])-1], 1000) # mask entity in the same column for prediction
        for e_i, (index, _) in enumerate(entities):
            tmp_j = 0
            for e_i_0 in column_en_map[index[1]]:
                if input_ent_local_id[2+e_i_0] != input_ent_local_id[2+e_i]:
                    exclusive_ent_mask[2+e_i,tmp_j] = input_ent_local_id[2+e_i_0]
                    tmp_j += 1

    else:
        input_ent_local_id = [find_id(pgEnt, all_entity_set) if pgEnt!=-1 else 0,0]
        i = 2
        while i < len(input_ent):
            if input_ent[i]>=len(RESERVED_ENT_VOCAB):
                e = input_ent[i]
                input_ent_local_id.append(find_id(e, all_entity_set))
                i+=1
            else:
                e = input_ent[i+1]
                tmp_e = find_id(e, all_entity_set)
                input_ent_local_id += [tmp_e, tmp_e]
                i+=2
        exclusive_ent_mask = None

    if len(core_entities) > 1:
        core_ent_local_id = [find_id(e, all_entity_set) for e in core_entities[1:]]
    else:
        core_ent_local_id = []
    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x
    return [table_id,np.array(input_tok),np.array(input_tok_type),np.array(input_tok_pos),(np.array(input_tok_mask[0]),np.array(input_tok_mask[1])),len(input_tok), \
                np.array(input_ent),input_ent_text_padded,input_ent_cell_length,np.array(input_ent_local_id),np.array(input_ent_type),(np.array(input_ent_mask[0]),np.array(input_ent_mask[1])),len(input_ent), \
                np.array(core_entity_mask),core_ent_local_id,all_entity_set,entity_cand,exclusive_ent_mask]

def process_single_hybrid_table_CER(input_table, config):
    table_id,subject,pgEnt,pgTitle,secTitle,caption,headers,core_entities,core_entities_text,all_entities,entities,entities_text,entity_cand = input_table
    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_header = config.tokenizer.encode(headers[0][1], max_length=config.max_header_length, add_special_tokens=False)
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    tokenized_header_length = len(tokenized_header)
    input_tok += tokenized_header
    input_tok_pos += list(range(tokenized_header_length))
    input_tok_type += [1]*tokenized_header_length

    input_ent = [entity for _, entity in entities]
    input_ent_text = [config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False) if len(entity_text)!=0 else [] for entity_text in entities_text]
    def find_id(e, e_list):
        for i, e_1 in enumerate(e_list):
            if e == e_1:
                return i
        # pdb.set_trace()
        raise Exception
    # prepend special token to input_ent, input_ent[0] = pgEnt, input_ent[1] = [CORE_ENT_MASK]
    input_ent_local_id = [find_id(e, core_entities) for e in input_ent]
    input_ent = [pgEnt if pgEnt!=-1 else 0, config.entity_wikid2id['[CORE_ENT_MASK]']] + input_ent
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length], []] + input_ent_text
    
    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x

    return [table_id,np.array(input_tok),np.array(input_tok_type),np.array(input_tok_pos),len(input_tok), \
                np.array(input_ent),input_ent_text_padded,input_ent_cell_length,len(input_ent),np.array(input_ent_local_id),np.array(core_entities), \
                entity_cand]

class WikiHybridTableDataset(Dataset):

    def _preprocess(self, data_dir):
        if self.mode == 0:
            preprocessed_filename = os.path.join(
                data_dir, "procressed_hybrid", self.src
            )
        elif self.mode == 1:
            preprocessed_filename = os.path.join(
                data_dir, "procressed_hybrid_CER", self.src
            )
        elif self.mode == 2:
            preprocessed_filename = os.path.join(
                data_dir, "procressed_hybrid_all", self.src
            )
        else:
            raise Exception
        preprocessed_filename += ".pickle"
        if not self.force_new and os.path.exists(preprocessed_filename):
            print("try loading preprocessed data from %s" % preprocessed_filename)
            with open(preprocessed_filename, "rb") as f:
                return pickle.load(f)
        else:
            print("try creating preprocessed data in %s" % preprocessed_filename)
            try:
                if self.mode == 0:
                    os.mkdir(os.path.join(data_dir, "procressed_hybrid"))
                elif self.mode == 1:
                    os.mkdir(os.path.join(data_dir, "procressed_hybrid_CER"))
                elif self.mode == 2:
                    os.mkdir(os.path.join(data_dir, "procressed_hybrid_all"))
                else:
                    raise Exception
            except FileExistsError:
                pass
            origin_data = open(os.path.join(data_dir, self.src + "_tables.jsonl"), "r")
            entity_candidate_file = os.path.join(data_dir, self.src + ".entity_candidate.pkl")
            if os.path.exists(entity_candidate_file):
                with open(entity_candidate_file, "rb") as f:
                    entity_candidate = pickle.load(f)
            else:
                entity_candidate = None
        print("Pre-processing data...")
        origin_table_num = 0
        actual_tables = []
        table_removed = 0
        for table in tqdm(origin_data):
            origin_table_num += 1
            table = json.loads(table.strip())
            table_id = table.get("_id","")
            pgTitle = table.get("pgTitle", "").lower()
            pgEnt = table.get("pgId", -1)
            if entity_candidate is not None:
                entity_cand = entity_candidate.get(table_id, [])
                entity_cand = [self.entity_wikid2id[z] for z in entity_cand if z in self.entity_wikid2id]
            else:
                entity_cand = None

            if pgEnt != -1:
                try:
                    pgEnt = self.entity_wikid2id[pgEnt]
                except:
                    pgEnt = -1
            secTitle = table.get("sectionTitle", "").lower()
            caption = table.get("tableCaption", "").lower()
            headers = table.get("processed_tableHeaders", [])
            rows = table.get("tableData", {})
            subject = table.get("subject_column", 0)
            entity_columns = table.get("entityColumn", [])
            headers = [[j, headers[j]] for j in entity_columns]
            entity_cells = np.array(table.get("entityCell",[[]]))
            core_entities = []
            core_entities_text = []
            all_entities = set()
            if pgEnt!=-1:
                all_entities.add(pgEnt)
            num_rows = len(rows)
            num_columns = len(rows[0])
            entities = []
            entities_text= []
            split = [0]
            tmp_entity_num = 0
            for i in range(num_rows):
                tmp_entities = []
                tmp_entities_text = []
                for j in range(num_columns):
                    if j in entity_columns:
                        if self.mode == 1 and j!=subject:
                            continue
                        if entity_cells[i,j] == 1:
                            try:
                                entity = self.entity_wikid2id[rows[i][j]['surfaceLinks'][0]['target']['id']]
                                entity_cells[i,j] = entity
                                tmp_entities.append([(i,j), entity])
                                tmp_entities_text.append(rows[i][j]['text'])
                            except:
                                entity_cells[i,j] = 0
                    else:
                        entity_cells[i,j] = 0
                if len(tmp_entities) == 0:
                    continue
                if self.mode == 0:
                    if i == 0 or not (entity_cells[i] == entity_cells[:i]).all(axis=1).any():
                        has_core = True if any([z[0][1]==subject for z in tmp_entities]) else False
                        if has_core or self.src == "train":
                            for (index, entity), entity_text in zip(tmp_entities, tmp_entities_text):
                                if self.mode == 0 and self.src != "train" and index[1]!=subject:
                                    entities.append([index, self.entity_wikid2id['[ENT_MASK]']])
                                    entities_text.append('')
                                    tmp_entity_num += 1
                                entities.append([index, entity])
                                entities_text.append(entity_text)
                                all_entities.add(entity)
                                tmp_entity_num += 1
                                if index[1] == subject:
                                    core_entities.append(entity)
                                    core_entities_text.append(entities_text)
                            if tmp_entity_num >= self.max_cell:
                                split.append(len(entities))
                                tmp_entity_num = 0
                elif self.mode == 1 or self.mode == 2:
                    for (index, entity), entity_text in zip(tmp_entities, tmp_entities_text):
                        entities.append([index, entity])
                        entities_text.append(entity_text)
                        tmp_entity_num += 1
                        core_entities.append(entity)
                        core_entities_text.append(entity_text)
                    if tmp_entity_num >= self.max_cell:
                        split.append(len(entities))
                        tmp_entity_num = 0
                #     pdb.set_trace()
            if split[-1]!=len(entities):
                split.append(len(entities))
            if len(core_entities) < 5:
                if self.mode!=2:
                    if self.src!="train" or len(core_entities) == 0 or (self.mode == 1 and len(core_entities) < 3):
                        table_removed += 1
                        continue
            if split[-2]!=0 and split[-1]-split[-2]<5:
                split[-2] = split[-1]
                split = split[:-1]
            for i in range(len(split)-1):
                actual_tables.append([
                    table_id,
                    subject,
                    pgEnt,
                    pgTitle,
                    secTitle,
                    caption,
                    headers,
                    core_entities,
                    core_entities_text,
                    all_entities,
                    entities[split[i]:split[i+1]],
                    entities_text[split[i]:split[i+1]],
                    entity_cand
                ])
            
        actual_table_num = len(actual_tables)
        print('%d original tables, actual %d tables in total\n%d tables removed because of extra entity filtering'%(origin_table_num, actual_table_num, table_removed))
        
        pool = Pool(processes=4)
        if self.mode == 0:
            processed_data = list(tqdm(pool.imap(partial(process_single_hybrid_table,config=self), actual_tables, chunksize=2000),total=len(actual_tables)))
        elif self.mode == 1 or self.mode == 2:
            processed_data = list(tqdm(pool.imap(partial(process_single_hybrid_table_CER,config=self), actual_tables, chunksize=2000),total=len(actual_tables)))
        # elif self.mode == 2:
        else:
            raise Exception
        pool.close()
        # pdb.set_trace()

        
        with open(preprocessed_filename, 'wb') as f:
            pickle.dump(processed_data, f)
        # pdb.set_trace()
        return processed_data

    def __init__(self, data_dir, entity_vocab, max_cell=100, max_input_tok=350, max_input_ent=150, src="train", max_length = [50, 10, 10], force_new=False, tokenizer = None, mode=0):
        if tokenizer is not None:
            self.tokenizer = tokenizer
        else:
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.src = src
        self.mode = mode #{0:pretrain,1:core entity retrieval,2:cell filling}
        self.max_cell = float(max_cell)
        self.max_title_length = max_length[0]
        self.max_header_length = max_length[1]
        self.max_cell_length = max_length[2]
        self.force_new = force_new
        self.max_input_tok = max_input_tok
        self.max_input_ent = max_input_ent
        self.entity_vocab = entity_vocab
        self.entity_wikid2id = {self.entity_vocab[x]['wiki_id']:x for x in self.entity_vocab}
        self.data = self._preprocess(data_dir)
        # pdb.set_trace()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def mask_ent(inputs_origin, inputs_local_id, core_entity_mask, entity_wikid2id, mlm_probability=0.15, mall_probability=0.5, is_train=False):
    """ Prepare masked entities inputs/labels for masked entity modeling: 80% MASK, 10% random, 10% original. """
    labels = inputs_local_id.clone()
    inputs = inputs_origin.clone()
    input_ent_mask_type = torch.zeros_like(inputs)
    if is_train:
        # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
        probability_matrix = torch.full(labels.shape, mlm_probability)
        special_tokens_mask = (inputs<len(RESERVED_ENT_VOCAB))#[list(map(lambda x: 1 if x == entity_wikid2id['[PAD]'] else 0, val)) for val in labels.tolist()]
        # special_tokens_mask[:, 1] = True
        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -1  # We only compute loss on masked tokens

        # 50% of the time, we mask input ent&text, the model need to recover the entity
        indices_mask_both = torch.bernoulli(torch.full(labels.shape, mall_probability)).bool() & masked_indices
        pg_ent_mask = torch.zeros(labels.shape)
        pg_ent_mask[:,0] = 1
        inputs[indices_mask_both] = 0
        input_ent_mask_type[indices_mask_both] = entity_wikid2id['[ENT_MASK]']
        input_ent_mask_type[indices_mask_both & pg_ent_mask.bool()] = entity_wikid2id['[PG_ENT_MASK]']
        input_ent_mask_type[indices_mask_both & core_entity_mask] = entity_wikid2id['[CORE_ENT_MASK]']
        # 50% of the time, we only mask ent, the task would be like entity linking
        # 40% of the time, we mask ent directly
        
        indices_mask_single = masked_indices & ~indices_mask_both
        inputs[indices_mask_single] = 0

        # 5% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.1)).bool() & indices_mask_single
        random_words = torch.randint(low=RESERVED_ENT_VOCAB_NUM,high=len(entity_wikid2id), size=labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        inputs[:, 1] = 0
        input_ent_mask_type[:, 1] = entity_wikid2id['[CORE_ENT_MASK]']
        indices_unchanged = torch.bernoulli(torch.full(labels.shape, 0.1)).bool() & masked_indices
        inputs[indices_unchanged] = inputs_origin[indices_unchanged]
        input_ent_mask_type[indices_unchanged] = 0

        # 10% of the time, we keep the masked input tokens unchanged

    else:
        ent_mask = inputs==entity_wikid2id['[ENT_MASK]']
        core_ent_mask = inputs==entity_wikid2id['[CORE_ENT_MASK]']
        labels[~ent_mask] = -1
        input_ent_mask_type[ent_mask] = entity_wikid2id['[ENT_MASK]']
        input_ent_mask_type[core_ent_mask] = entity_wikid2id['[CORE_ENT_MASK]']
        inputs[ent_mask|core_ent_mask] = 0

    return inputs, input_ent_mask_type, labels
class pretrain_hybrid_table_collate_fn:
    def __init__(self, tokenizer, entity_wikid2id, mlm_probability, ent_mlm_probability, mall_probability=0.5, max_entity_candidate=1000, is_train=True, candidate_distribution=None, use_cand=True, random_sample=True, use_visibility=True):
        self.tokenizer = tokenizer
        self.entity_wikid2id = entity_wikid2id
        self.mlm_probability = mlm_probability
        self.ent_mlm_probability = ent_mlm_probability
        self.mall_probability = mall_probability
        self.max_entity_candidate = max_entity_candidate
        self.is_train = is_train
        self.candidate_distribution = candidate_distribution
        self.use_cand = use_cand
        self.random_sample = random_sample
        self.use_visibility = use_visibility

    def generate_random_candidate(self, batch_size, indice_mask):
        random_shifts = np.random.random((batch_size, len(self.entity_wikid2id)))
        random_shifts[indice_mask] = 10
        return np.argpartition(random_shifts, self.max_entity_candidate, axis=1)[:, :self.max_entity_candidate]
    
    def generate_random_candidate_v2(self, batch_size, masked_entity, candidate_distribution=None, ent_cand=None):
        if self.random_sample and self.is_train:
            random_shifts = np.random.random(len(self.entity_wikid2id))
            if candidate_distribution is not None:
                random_shifts /= np.sum(random_shifts)
                random_shifts -= candidate_distribution
            all_masked = list(itertools.chain(*masked_entity))
            random_shifts[all_masked] = 10
            all_masked = set(all_masked)
            final_candidates = np.tile(np.argpartition(random_shifts, self.max_entity_candidate)[:self.max_entity_candidate],[batch_size, 1])
        else:
            final_candidates = np.zeros([batch_size, self.max_entity_candidate])
        for i, masked in enumerate(masked_entity):
            final_candidates[i, :len(masked)] = masked
            if self.use_cand:
                cand_i = ent_cand[i]
                if self.random_sample and self.is_train:
                    if len(cand_i)+len(masked) > self.max_entity_candidate/2:
                        cand_i = random.sample(cand_i, int(self.max_entity_candidate/2-len(masked)))
                else:
                    if len(cand_i)+len(masked) > self.max_entity_candidate:
                        cand_i = random.sample(cand_i, int(self.max_entity_candidate-len(masked)))
                final_candidates[i, len(masked):len(masked)+len(cand_i)] = cand_i
            else:
                remain = list(all_masked-set(masked))
                final_candidates[i, len(masked):len(masked)+len(remain)] = remain
        return final_candidates
        
    def __call__(self, raw_batch):
        batch_table_id,batch_input_tok,batch_input_tok_type,batch_input_tok_pos,batch_input_tok_mask,batch_input_tok_length, \
            batch_input_ent,batch_input_ent_text,batch_input_ent_cell_length,batch_input_ent_local_id,batch_input_ent_type,batch_input_ent_mask,batch_input_ent_length, \
            batch_core_entity_mask,batch_core_ent_local_id,batch_all_entity_set,batch_entity_cand,batch_exclusive_ent_mask = zip(*raw_batch)
        
        if batch_entity_cand[0] is None and self.use_cand:
            raise Exception

        max_input_tok_length = max(batch_input_tok_length)
        max_input_ent_length = max(batch_input_ent_length)
        max_input_cell_length = max([z.shape[-1] for z in batch_input_ent_text])
        batch_size = len(batch_input_tok_length)

        batch_input_tok_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_type_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_pos_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_mask_padded = np.zeros([batch_size, max_input_tok_length, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_input_ent_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_text_padded = np.zeros([batch_size, max_input_ent_length, max_input_cell_length], dtype=int)
        batch_input_ent_text_length = np.ones([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_type_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_mask_padded = np.zeros([batch_size, max_input_ent_length, max_input_tok_length+max_input_ent_length], dtype=int)
        batch_core_entity_mask_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_local_id_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        if self.is_train:
            max_input_col_ent_num = max([z.shape[1] for z in batch_exclusive_ent_mask])
            batch_exclusive_ent_mask_padded = np.full([batch_size, max_input_ent_length, max_input_col_ent_num], 1000, dtype=int)

        for i, (tok_l, ent_l) in enumerate(zip(batch_input_tok_length, batch_input_ent_length)):
            batch_input_tok_padded[i, :tok_l] = batch_input_tok[i]
            batch_input_tok_type_padded[i, :tok_l] = batch_input_tok_type[i]
            batch_input_tok_pos_padded[i, :tok_l] = batch_input_tok_pos[i]
            if self.use_visibility or not self.is_train:
                batch_input_tok_mask_padded[i, :tok_l, :tok_l] = batch_input_tok_mask[i][0]
                batch_input_tok_mask_padded[i, :tok_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_tok_mask[i][1]
            else:
                batch_input_tok_mask_padded[i, :tok_l, :tok_l] = 1
                batch_input_tok_mask_padded[i, :tok_l, max_input_tok_length:max_input_tok_length+ent_l] = 1
            batch_input_ent_padded[i, :ent_l] = batch_input_ent[i]
            batch_input_ent_text_padded[i, :ent_l, :batch_input_ent_text[i].shape[-1]] = batch_input_ent_text[i]
            batch_input_ent_text_length[i, :ent_l] = batch_input_ent_cell_length[i]
            batch_input_ent_type_padded[i, :ent_l] = batch_input_ent_type[i]
            if self.use_visibility or not self.is_train:
                batch_input_ent_mask_padded[i, :ent_l, :tok_l] = batch_input_ent_mask[i][0]
                batch_input_ent_mask_padded[i, :ent_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_ent_mask[i][1]
            else:
                batch_input_ent_mask_padded[i, :ent_l, :tok_l] = 1
                batch_input_ent_mask_padded[i, :ent_l, max_input_tok_length:max_input_tok_length+ent_l] = 1
            batch_core_entity_mask_padded[i, :ent_l] = batch_core_entity_mask[i]
            batch_input_ent_local_id_padded[i, :ent_l] = batch_input_ent_local_id[i]
            if self.is_train:
                batch_exclusive_ent_mask_padded[i, :ent_l, :batch_exclusive_ent_mask[i].shape[1]] = batch_exclusive_ent_mask[i]
        
        batch_input_tok_padded = torch.LongTensor(batch_input_tok_padded)
        batch_input_tok_type_padded = torch.LongTensor(batch_input_tok_type_padded)
        batch_input_tok_pos_padded = torch.LongTensor(batch_input_tok_pos_padded)
        batch_input_tok_mask_padded = torch.LongTensor(batch_input_tok_mask_padded)

        batch_input_ent_padded = torch.LongTensor(batch_input_ent_padded)
        batch_input_ent_text_padded = torch.LongTensor(batch_input_ent_text_padded)
        batch_input_ent_text_length = torch.LongTensor(batch_input_ent_text_length)
        batch_input_ent_type_padded = torch.LongTensor(batch_input_ent_type_padded)
        batch_input_ent_mask_padded = torch.LongTensor(batch_input_ent_mask_padded)
        batch_core_entity_mask_padded = torch.BoolTensor(batch_core_entity_mask_padded)
        batch_input_ent_local_id_padded = torch.LongTensor(batch_input_ent_local_id_padded)
        if self.is_train:
            batch_exclusive_ent_mask_padded = torch.LongTensor(batch_exclusive_ent_mask_padded)
        else:
            batch_exclusive_ent_mask_padded = None

        batch_input_tok_final, batch_input_tok_labels = mask_tokens(batch_input_tok_padded, self.tokenizer, mlm_probability=self.mlm_probability)
        batch_input_ent_final, batch_input_ent_mask_type, batch_input_ent_labels = mask_ent(batch_input_ent_padded, batch_input_ent_local_id_padded, batch_core_entity_mask_padded, self.entity_wikid2id, mlm_probability=self.ent_mlm_probability, mall_probability=self.mall_probability, is_train=self.is_train)

        #random sample candidate
        # indice_mask = (list(itertools.chain(*[[i]*len(x) for i,x in enumerate(batch_all_entity_set)])), list(itertools.chain(*batch_all_entity_set)))
        # batch_candidate_entity_set = self.generate_random_candidate(batch_size, indice_mask)
        batch_candidate_entity_set = self.generate_random_candidate_v2(batch_size, batch_all_entity_set, self.candidate_distribution, ent_cand=batch_entity_cand)
        batch_candidate_entity_set = torch.LongTensor(batch_candidate_entity_set)
        if not self.is_train:
            batch_core_entity_label = np.zeros([batch_size, self.max_entity_candidate], dtype=bool)
            for i in range(batch_size):
                batch_core_entity_label[i, batch_core_ent_local_id[i]] = True
            batch_core_entity_label = torch.BoolTensor(batch_core_entity_label)
        else:
            batch_core_entity_label = None
        return batch_table_id,batch_input_tok_final, batch_input_tok_type_padded, batch_input_tok_pos_padded, batch_input_tok_labels, batch_input_tok_mask_padded, \
                batch_input_ent_text_padded, batch_input_ent_text_length, batch_input_ent_mask_type, batch_input_ent_final, batch_input_ent_type_padded, batch_input_ent_labels, batch_input_ent_mask_padded, batch_candidate_entity_set, batch_core_entity_label, batch_exclusive_ent_mask_padded, batch_core_entity_mask_padded

class pretrain_hybrid_table_collate_fn_CER(pretrain_hybrid_table_collate_fn):
    def __init__(self, tokenizer, entity_wikid2id, max_entity_candidate=1000, is_train=True, candidate_distribution=None, use_cand=True, seed_num=1, random_sample=True):
        self.tokenizer = tokenizer
        self.entity_wikid2id = entity_wikid2id
        self.max_entity_candidate = max_entity_candidate
        self.is_train = is_train
        self.candidate_distribution = candidate_distribution
        self.use_cand = use_cand
        self.seed = seed_num
        self.random_sample = random_sample
    def __call__(self, raw_batch):
        batch_table_id,batch_input_tok,batch_input_tok_type,batch_input_tok_pos,batch_input_tok_length, \
            batch_input_ent,batch_input_ent_text,batch_input_ent_cell_length,batch_input_ent_length,batch_input_ent_local_id,batch_core_entities, \
            batch_entity_cand = zip(*raw_batch)
        
        if batch_entity_cand[0] is None and self.use_cand:
            raise Exception

        max_input_tok_length = max(batch_input_tok_length)
        max_input_ent_length = max(batch_input_ent_length)
        max_input_cell_length = max([z.shape[-1] for z in batch_input_ent_text])
        batch_size = len(batch_input_tok_length)

        batch_input_tok_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_type_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_pos_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)

        batch_input_ent_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_text_padded = np.zeros([batch_size, max_input_ent_length, max_input_cell_length], dtype=int)
        batch_input_ent_text_length = np.ones([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_type_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_type_padded[:, 0] = 2
        batch_input_ent_ent_mask_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_ent_mask_padded[:, 1] = 1

        batch_input_mask_padded = np.zeros([batch_size, 1, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_seed_ent = []
        batch_target_ent = np.full([batch_size, self.max_entity_candidate], 0, dtype=int)
        for i, (tok_l, ent_l) in enumerate(zip(batch_input_tok_length, batch_input_ent_length)):
            batch_input_tok_padded[i, :tok_l] = batch_input_tok[i]
            batch_input_tok_type_padded[i, :tok_l] = batch_input_tok_type[i]
            batch_input_tok_pos_padded[i, :tok_l] = batch_input_tok_pos[i]
            batch_input_ent_padded[i, :ent_l] = batch_input_ent[i]
            batch_input_ent_text_padded[i, :ent_l, :batch_input_ent_text[i].shape[-1]] = batch_input_ent_text[i]
            batch_input_ent_text_length[i, :ent_l] = batch_input_ent_cell_length[i]
            batch_input_ent_type_padded[i, 1:ent_l] = 3

            if self.seed >0:
                tmp_cand_core = set(range(ent_l-2))
                tmp_selected_core = random.sample(tmp_cand_core,self.seed)
                batch_seed_ent.append(batch_input_ent_local_id[i][tmp_selected_core])
                tmp_cand_core = list(tmp_cand_core-set(tmp_selected_core))
                # batch_target_ent[i,:len(tmp_cand_core)] = batch_input_ent_local_id[i][tmp_cand_core]
                batch_target_ent[i,batch_input_ent_local_id[i][tmp_cand_core]] = 1
                batch_input_ent_ent_mask_padded[i,2:][tmp_selected_core] = 1
            elif self.seed == 0:
                batch_input_ent_ent_mask_padded[i,2:ent_l] = 0
                batch_seed_ent.append([])
                batch_target_ent[i,batch_input_ent_local_id[i]] = 1
            else:
                batch_input_ent_ent_mask_padded[i,2:ent_l] = 1
            batch_input_ent_ent_mask_padded[i,0] = batch_input_ent[i][0]!=0

            batch_input_mask_padded[i, :, :tok_l] = 1

        batch_input_mask_padded[:,0,max_input_tok_length:] = batch_input_ent_ent_mask_padded        

        
        batch_input_tok_padded = torch.LongTensor(batch_input_tok_padded)
        batch_input_tok_type_padded = torch.LongTensor(batch_input_tok_type_padded)
        batch_input_tok_pos_padded = torch.LongTensor(batch_input_tok_pos_padded)

        batch_input_ent_padded = torch.LongTensor(batch_input_ent_padded)
        batch_input_ent_text_padded = torch.LongTensor(batch_input_ent_text_padded)
        batch_input_ent_text_length = torch.LongTensor(batch_input_ent_text_length)
        batch_input_ent_type_padded = torch.LongTensor(batch_input_ent_type_padded)

        batch_input_mask_padded = torch.LongTensor(batch_input_mask_padded)
        batch_seed_ent = torch.LongTensor(batch_seed_ent)
        batch_target_ent = torch.LongTensor(batch_target_ent)

        #random sample candidate
        batch_candidate_entity_set = self.generate_random_candidate_v2(batch_size, batch_core_entities, self.candidate_distribution, ent_cand=batch_entity_cand)
        batch_candidate_entity_set = torch.LongTensor(batch_candidate_entity_set)
        return batch_table_id,batch_input_tok_padded, batch_input_tok_type_padded, batch_input_tok_pos_padded, \
                batch_input_ent_padded, batch_input_ent_text_padded, batch_input_ent_text_length, batch_input_ent_type_padded, \
                batch_input_mask_padded, batch_candidate_entity_set, batch_seed_ent, batch_target_ent

def CER_build_input(pgEnt, pgTitle, secTitle, caption, header, seed_entities, seed_entities_text, entity_cand, config):
    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_header = config.tokenizer.encode(header, max_length=config.max_header_length, add_special_tokens=False)
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    tokenized_header_length = len(tokenized_header)
    input_tok += tokenized_header
    input_tok_pos += list(range(tokenized_header_length))
    input_tok_type += [1]*tokenized_header_length

    input_ent = [config.entity_wikid2id[entity] for entity in seed_entities]
    input_ent_text = [config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False) if len(entity_text)!=0 else [] for entity_text in seed_entities_text]
    input_ent = [config.entity_wikid2id[pgEnt] if pgEnt!=-1 else 0, config.entity_wikid2id['[CORE_ENT_MASK]']] + input_ent
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length], []] + input_ent_text
    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x

    input_mask = np.ones([1, 1, len(input_tok)+len(input_ent)], dtype=int)
    if input_ent[0]==0:
        input_mask[0,:,len(input_tok)] = 0
    input_mask = torch.LongTensor(input_mask)

    input_tok = torch.LongTensor([input_tok])
    input_tok_type = torch.LongTensor([input_tok_type])
    input_tok_pos = torch.LongTensor([input_tok_pos])
    
    input_ent = torch.LongTensor([input_ent])
    input_ent_text = torch.LongTensor([input_ent_text_padded])
    input_ent_cell_length = torch.LongTensor([input_ent_cell_length])
    input_ent_type = torch.full_like(input_ent, 3)
    input_ent_type[:, 0] = 2

    candidate_entity_set = [config.entity_wikid2id[entity] for entity in entity_cand]
    candidate_entity_set = torch.LongTensor([candidate_entity_set])

    return input_tok, input_tok_type, input_tok_pos, input_mask,\
            input_ent, input_ent_text, input_ent_cell_length, input_ent_type, candidate_entity_set

def CF_build_input(pgEnt, pgTitle, secTitle, caption, headers, core_entities, core_entities_text, entity_cand, config):
    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_headers = [config.tokenizer.encode(header, max_length=config.max_header_length, add_special_tokens=False) for header in headers]
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    header_span = []
    for tokenized_header in tokenized_headers:
        tokenized_header_length = len(tokenized_header)
        header_span.append([len(input_tok), len(input_tok)+tokenized_header_length])
        input_tok += tokenized_header
        input_tok_pos += list(range(tokenized_header_length))
        input_tok_type += [1]*tokenized_header_length

    input_ent = [config.entity_wikid2id[pgEnt] if pgEnt!=-1 else 0]
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length]]
    input_ent_type = [2]

    input_ent += [config.entity_wikid2id[entity] for entity in core_entities]
    input_ent_text += [config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False) if len(entity_text)!=0 else [] for entity_text in core_entities_text]
    input_ent_type += [3]*len(core_entities)

    input_ent += [config.entity_wikid2id['[ENT_MASK]']]*len(core_entities)
    input_ent_text += [[]]*len(core_entities)
    input_ent_type += [4]*len(core_entities)

    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x
    assert len(input_ent) == 1+2*len(core_entities)

    input_tok_mask = np.ones([1, len(input_tok), len(input_tok)+len(input_ent)], dtype=int)
    input_tok_mask[0, header_span[0][0]:header_span[0][1], len(input_tok)+1+len(core_entities):] = 0
    input_tok_mask[0, header_span[1][0]:header_span[1][1], len(input_tok)+1:len(input_tok)+1+len(core_entities)] = 0
    input_tok_mask[0, :, len(input_tok)+1+len(core_entities):] = 0

    input_ent_mask = np.ones([1, len(input_ent), len(input_tok)+len(input_ent)], dtype=int)
    input_ent_mask[0, 1:1+len(core_entities), header_span[1][0]:header_span[1][1]] = 0
    input_ent_mask[0, 1:1+len(core_entities), len(input_tok)+1+len(core_entities):] = np.eye(len(core_entities), dtype=int)
    # input_ent_mask[0, 1:1+len(core_entities), len(input_tok)+1:len(input_tok)+1+len(core_entities)] = np.eye(len(core_entities), dtype=int)
    input_ent_mask[0, 1+len(core_entities):, header_span[0][0]:header_span[0][1]] = 0
    input_ent_mask[0, 1+len(core_entities):, len(input_tok)+1:len(input_tok)+1+len(core_entities)] = np.eye(len(core_entities), dtype=int)
    input_ent_mask[0, 1+len(core_entities):, len(input_tok)+1+len(core_entities):] = np.eye(len(core_entities), dtype=int)

    input_tok_mask = torch.LongTensor(input_tok_mask)
    input_ent_mask = torch.LongTensor(input_ent_mask)

    input_tok = torch.LongTensor([input_tok])
    input_tok_type = torch.LongTensor([input_tok_type])
    input_tok_pos = torch.LongTensor([input_tok_pos])
    
    input_ent = torch.LongTensor([input_ent])
    input_ent_text = torch.LongTensor([input_ent_text_padded])
    input_ent_cell_length = torch.LongTensor([input_ent_cell_length])
    input_ent_type = torch.LongTensor([input_ent_type])

    input_ent_mask_type = torch.zeros_like(input_ent)
    input_ent_mask_type[:,:,1+len(core_entities):] = config.entity_wikid2id['[ENT_MASK]']

    candidate_entity_set = [config.entity_wikid2id[entity] for entity in entity_cand]
    candidate_entity_set = torch.LongTensor([candidate_entity_set])
    

    return input_tok, input_tok_type, input_tok_pos, input_tok_mask,\
            input_ent, input_ent_text, input_ent_cell_length, input_ent_type, input_ent_mask_type, input_ent_mask, candidate_entity_set


class HybridTableLoader(DataLoader):
    """
    Base class for all data loaders
    """
    def __init__(
        self,
        dataset,
        batch_size,
        max_entity_candidate=1000,
        sample_distribution=None,
        shuffle=True,
        is_train = True,
        num_workers=0,
        mlm_probability=0.15,
        ent_mlm_probability=0.15,
        mall_probability=0.5,
        sampler=None,
        use_cand=True,
        mode=0,
        random_sample = True,
        seed_num = 1,
        use_visibility = True
    ):
        self.mlm_probability = mlm_probability
        self.ent_mlm_probability = ent_mlm_probability
        self.mall_probability = mall_probability
        self.max_entity_candidate=max_entity_candidate
        self.shuffle = shuffle
        self.use_cand = use_cand
        self.random_sample = random_sample
        if sampler is not None:
            self.shuffle = False

        self.batch_idx = 0
        self.n_samples = len(dataset)
        self.is_train = is_train
        self.sample_distribution = sample_distribution
        self.mode = mode
        if self.mode == 0:
            self.collate_fn = pretrain_hybrid_table_collate_fn(dataset.tokenizer, dataset.entity_wikid2id, self.mlm_probability, self.ent_mlm_probability, self.mall_probability, self.max_entity_candidate, is_train=self.is_train, candidate_distribution=self.sample_distribution, use_cand=self.use_cand, random_sample=self.random_sample, use_visibility=use_visibility)
        elif self.mode == 1:
            self.collate_fn = pretrain_hybrid_table_collate_fn_CER(dataset.tokenizer, dataset.entity_wikid2id, self.max_entity_candidate, is_train=self.is_train, candidate_distribution=self.sample_distribution, use_cand=self.use_cand, random_sample=self.random_sample, seed_num=seed_num)
        else:
            raise Exception

        self.init_kwargs = {
            "dataset": dataset,
            "batch_size": batch_size,
            "shuffle": self.shuffle,
            "collate_fn": self.collate_fn,
            "num_workers": num_workers,
            "sampler": sampler
        }
        super().__init__(**self.init_kwargs)

**Making changes to py file hybrid_data_loaders. This script uses a bert uncase model which was perhaps developed by them . However that is not something that is in the data file even if we download them. Luckily Hugging face has a bert-uncase model that works fine here!!!**

In [ ]:
dataset = WikiHybridTableDataset(data_dir,entity_vocab,max_cell=100, max_input_tok=350, max_input_ent=150, src="dev", max_length = [50, 10, 10], force_new=False, tokenizer = None, mode=0)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

try loading preprocessed data from /content/drive/MyDrive/TURL/data/procressed_hybrid/dev.pickle


In [ ]:
# This is an example of converting an arbitrary table to input
# Here we show an example for cell filling task
# The input entites are entities in the subject column, we append [ENT_MASK] and use its representation to match with the candidate entities
def CF_build_input(pgEnt, pgTitle, secTitle, caption, headers, core_entities, core_entities_text, entity_cand, config):
    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_headers = [config.tokenizer.encode(header, max_length=config.max_header_length, add_special_tokens=False) for header in headers]
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    header_span = []
    for tokenized_header in tokenized_headers:
        tokenized_header_length = len(tokenized_header)
        header_span.append([len(input_tok), len(input_tok)+tokenized_header_length])
        input_tok += tokenized_header
        input_tok_pos += list(range(tokenized_header_length))
        input_tok_type += [1]*tokenized_header_length

    input_ent = [config.entity_wikid2id[pgEnt] if pgEnt!=-1 else 0]
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length]]
    input_ent_type = [2]
    
    # core entities in the subject column
    input_ent += [config.entity_wikid2id[entity] for entity in core_entities]
    input_ent_text += [config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False) if len(entity_text)!=0 else [] for entity_text in core_entities_text]
    input_ent_type += [3]*len(core_entities)
    
    # append [ent_mask]
    input_ent += [config.entity_wikid2id['[ENT_MASK]']]*len(core_entities)
    input_ent_text += [[]]*len(core_entities)
    input_ent_type += [4]*len(core_entities)

    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x
    assert len(input_ent) == 1+2*len(core_entities)

    input_tok_mask = np.ones([1, len(input_tok), len(input_tok)+len(input_ent)], dtype=int)
    input_tok_mask[0, header_span[0][0]:header_span[0][1], len(input_tok)+1+len(core_entities):] = 0
    input_tok_mask[0, header_span[1][0]:header_span[1][1], len(input_tok)+1:len(input_tok)+1+len(core_entities)] = 0
    input_tok_mask[0, :, len(input_tok)+1+len(core_entities):] = 0

    # build the mask for entities
    input_ent_mask = np.ones([1, len(input_ent), len(input_tok)+len(input_ent)], dtype=int)
    input_ent_mask[0, 1:1+len(core_entities), header_span[1][0]:header_span[1][1]] = 0
    input_ent_mask[0, 1:1+len(core_entities), len(input_tok)+1+len(core_entities):] = np.eye(len(core_entities), dtype=int)
    input_ent_mask[0, 1+len(core_entities):, header_span[0][0]:header_span[0][1]] = 0
    input_ent_mask[0, 1+len(core_entities):, len(input_tok)+1:len(input_tok)+1+len(core_entities)] = np.eye(len(core_entities), dtype=int)
    input_ent_mask[0, 1+len(core_entities):, len(input_tok)+1+len(core_entities):] = np.eye(len(core_entities), dtype=int)

    input_tok_mask = torch.LongTensor(input_tok_mask)
    input_ent_mask = torch.LongTensor(input_ent_mask)

    input_tok = torch.LongTensor([input_tok])
    input_tok_type = torch.LongTensor([input_tok_type])
    input_tok_pos = torch.LongTensor([input_tok_pos])
    
    input_ent = torch.LongTensor([input_ent])
    input_ent_text = torch.LongTensor([input_ent_text_padded])
    input_ent_cell_length = torch.LongTensor([input_ent_cell_length])
    input_ent_type = torch.LongTensor([input_ent_type])

    input_ent_mask_type = torch.zeros_like(input_ent)
    input_ent_mask_type[:,1+len(core_entities):] = config.entity_wikid2id['[ENT_MASK]']

    candidate_entity_set = [config.entity_wikid2id[entity] for entity in entity_cand]
    candidate_entity_set = torch.LongTensor([candidate_entity_set])
    

    return input_tok, input_tok_type, input_tok_pos, input_tok_mask,\
            input_ent, input_ent_text, input_ent_cell_length, input_ent_type, input_ent_mask_type, input_ent_mask, candidate_entity_set

**The cell below is edited. Running the original cell gives the following errorr:**

***RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)***

In [ ]:
results = []
for table_id,pgEnt,pgTitle,secTitle,caption,(h1, h2),data_sample in tqdm(dev_data):
    result = []
    while len(data_sample)!=0:
        core_entities = []
        core_entities_text = []
        target_entities = []
        all_entity_cand = set()
        entity_cand = []
        for (core_e, core_e_text), target_e in data_sample[:100]:
            assert target_e in entity_wikid2id
            core_entities.append(core_e)
            core_entities_text.append(core_e_text)
            target_entities.append(target_e)
            cands = CF.get_cand_row(core_e, h2)
            cands = {key:value for key,value in cands.items() if key in entity_wikid2id}
            entity_cand.append(cands)
            all_entity_cand |= set(cands.keys()) 
        all_entity_cand = list(all_entity_cand)
        input_tok, input_tok_type, input_tok_pos, input_tok_mask,input_ent, input_ent_text, input_ent_text_length, input_ent_type, input_ent_mask_type, input_ent_mask,candidate_entity_set = CF_build_input(pgEnt, pgTitle, secTitle, caption, [h1, h2], core_entities, core_entities_text, all_entity_cand, dataset)
        input_tok = input_tok.to(device)
        input_tok_type = input_tok_type.to(device)
        input_tok_pos = input_tok_pos.to(device)
        input_tok_mask = input_tok_mask.to(device)
        input_ent_text = input_ent_text.to(device)
        input_ent_text_length = input_ent_text_length.to(device)
        input_ent = input_ent.to(device)
        input_ent_type = input_ent_type.to(device)
        input_ent_mask_type = input_ent_mask_type.to(device)
        input_ent_mask = input_ent_mask.to(device)
        candidate_entity_set = candidate_entity_set.to(device)

        # added line
        #-------------------------

        model.to(device=device)

        #------------------------
        with torch.no_grad():
            tok_outputs, ent_outputs = model(input_tok, input_tok_type, input_tok_pos, input_tok_mask,
                            input_ent_text, input_ent_text_length, input_ent_mask_type,input_ent, input_ent_type, input_ent_mask, candidate_entity_set)
            num_sample = len(target_entities)
            ent_prediction_scores = ent_outputs[0][0,num_sample+1:].tolist()
        for i, target_e in enumerate(target_entities):
            predictions = ent_prediction_scores[i]
            if len(entity_cand[i]) == 0:
                result.append([target_e, entity_cand[i], [], []])
            else:
                tmp_cand_scores = []
                for j, cand_e in enumerate(all_entity_cand):
                    if cand_e in entity_cand[i]:
                        tmp_cand_scores.append([cand_e, predictions[j]])
                sorted_cand_scores =  sorted(tmp_cand_scores, key=lambda z:z[1], reverse=True)
                sorted_cands = [z[0] for z in sorted_cand_scores]
                # use H2H as baseline
                base_sorted_cands = CF.rank_cand_h2h(h2, entity_cand[i])
                result.append([target_e, entity_cand[i], sorted_cands, base_sorted_cands])
        data_sample = data_sample[100:]
    results.append({
        'pgTitle': pgTitle,
        'secTitle': secTitle,
        'caption': caption,
        'headers': [h1, h2],
        'result': result
    })

  0%|          | 0/9075 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
100%|██████████| 9075/9075 [05:04<00:00, 29.78it/s]


In [ ]:
print('tok(metadata) outputs', len(tok_outputs))
print('tok prediction logits: [batch_size, num_toks, vocab_size]\n', tok_outputs[0].shape)
print('tok hidden states: [batch_size, num_toks, hidden_size]\n', tok_outputs[1].shape)
print('tok attention: n_layers*[batch_size, num_attention_headers, num_toks, num_toks+num_ents]\n', tok_outputs[2][0].shape)
print('entity(cell) outputs', len(ent_outputs))
print('ent prediction logits: [batch_size, num_ents, candidate_size]\n', ent_outputs[0].shape)
print('ent hidden states: [batch_size, num_ents, hidden_size]\n', ent_outputs[1].shape)
print('ent attention: n_layers*[batch_size, num_attention_headers, num_ents, num_toks+num_ents]\n', ent_outputs[2][0].shape)

tok(metadata) outputs 3
tok prediction logits: [batch_size, num_toks, vocab_size]
 torch.Size([1, 10, 30522])
tok hidden states: [batch_size, num_toks, hidden_size]
 torch.Size([1, 10, 312])
tok attention: n_layers*[batch_size, num_attention_headers, num_toks, num_toks+num_ents]
 torch.Size([1, 12, 10, 19])
entity(cell) outputs 3
ent prediction logits: [batch_size, num_ents, candidate_size]
 torch.Size([1, 9, 53])
ent hidden states: [batch_size, num_ents, hidden_size]
 torch.Size([1, 9, 312])
ent attention: n_layers*[batch_size, num_attention_headers, num_ents, num_toks+num_ents]
 torch.Size([1, 12, 9, 19])


In [ ]:
def get_precision(result):
    recall = 0
    precision_neural = [0, 0, 0, 0]
    precision_base = [0, 0, 0, 0]
    for target_e, cand, p_neural, p_base in result:
        if target_e in cand:
            recall += 1
            if target_e == p_neural[0]:
                precision_neural[0] += 1
            if target_e == p_base[0]:
                precision_base[0] += 1
            if target_e in p_neural[:3]:
                precision_neural[1] += 1
            if target_e in p_neural[:5]:
                precision_neural[2] += 1
            if target_e in p_neural[:10]:
                precision_neural[3] += 1
            if target_e in p_base[:3]:
                precision_base[1] += 1
            if target_e in p_base[:5]:
                precision_base[2] += 1
            if target_e in p_base[:10]:
                precision_base[3] += 1
    if recall != 0:
        return recall/len(result), [z/recall for z in precision_neural], [z/recall for z in precision_base]
    else:
        return 0, [0 for z in precision_neural], [0 for z in precision_base]

In [ ]:
final_results = [get_precision(x['result']) for x in results]
print('recall', np.mean([x[0] for x in final_results]))
print('neural')
print('p@1', np.mean([x[1][0] for x in final_results if x[0]!=0]))
print('p@3', np.mean([x[1][1] for x in final_results if x[0]!=0]))
print('p@5', np.mean([x[1][2] for x in final_results if x[0]!=0]))
print('p@10', np.mean([x[1][3] for x in final_results if x[0]!=0]))
print('base')
print('p@1', np.mean([x[2][0] for x in final_results if x[0]!=0]))
print('p@3', np.mean([x[2][1] for x in final_results if x[0]!=0]))
print('p@5', np.mean([x[2][2] for x in final_results if x[0]!=0]))
print('p@10', np.mean([x[2][3] for x in final_results if x[0]!=0]))

recall 0.6145301312480205
neural
p@1 0.23121257618176175
p@3 0.4530341949269496
p@5 0.5483342080901582
p@10 0.6678285388664684
base
p@1 0.5189550699035596
p@3 0.7094590968795172
p@5 0.7733420876389275
p@10 0.8544311158445016


<a class="anchor" id="el"></a>
# EL
Evaluate Entity Linking

In [ ]:
final_results = [get_precision(x['result']) for x in results]
print('recall', np.mean([x[0] for x in final_results]))
print('neural')
print('p@1', np.mean([x[1][0] for x in final_results if x[0]!=0]))
print('p@3', np.mean([x[1][1] for x in final_results if x[0]!=0]))
print('p@5', np.mean([x[1][2] for x in final_results if x[0]!=0]))
print('p@10', np.mean([x[1][3] for x in final_results if x[0]!=0]))
print('base')
print('p@1', np.mean([x[2][0] for x in final_results if x[0]!=0]))
print('p@3', np.mean([x[2][1] for x in final_results if x[0]!=0]))
print('p@5', np.mean([x[2][2] for x in final_results if x[0]!=0]))
print('p@10', np.mean([x[2][3] for x in final_results if x[0]!=0]))

recall 0.6145301312480205
neural
p@1 0.23121257618176175
p@3 0.4530341949269496
p@5 0.5483342080901582
p@10 0.6678285388664684
base
p@1 0.5189550699035596
p@3 0.7094590968795172
p@5 0.7733420876389275
p@10 0.8544311158445016


In [ ]:
print('neural')
print('p@1', np.mean([x[1][0] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@3', np.mean([x[1][1] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@5', np.mean([x[1][2] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@10', np.mean([x[1][3] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('base')
print('p@1', np.mean([x[2][0] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@3', np.mean([x[2][1] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@5', np.mean([x[2][2] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@10', np.mean([x[2][3] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))

neural
p@1 0.24136023957323663
p@3 0.46718400585218456
p@5 0.56236821678784
p@10 0.6823428781448035
base
p@1 0.5391679272617237
p@3 0.7275446853176161
p@5 0.7879552266125807
p@10 0.8620558693826613


In [ ]:
print('neural')
print('p@1', np.mean([x[1][0] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@3', np.mean([x[1][1] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@5', np.mean([x[1][2] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@10', np.mean([x[1][3] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('base')
print('p@1', np.mean([x[2][0] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@3', np.mean([x[2][1] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@5', np.mean([x[2][2] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))
print('p@10', np.mean([x[2][3] for i,x in enumerate(final_results) if x[0]!=0 and 'team' not in results[i]['headers'][1]]))

neural
p@1 0.24136023957323663
p@3 0.46718400585218456
p@5 0.56236821678784
p@10 0.6823428781448035
base
p@1 0.5391679272617237
p@3 0.7275446853176161
p@5 0.7879552266125807
p@10 0.8620558693826613


In [ ]:
#load dbpedia types from depedia_type_vocab.txt
type_vocab = load_dbpedia_type_vocab(data_dir)
config_class, model_class, _ = MODEL_CLASSES['EL']
config = config_class.from_pretrained(config_name)
config.ent_type_vocab_size = len(type_vocab)
config.mode=0

In [ ]:
with open(os.path.join(data_dir, 'test_own.table_entity_linking.json'), 'r') as f:
    example = json.load(f)[0]
display(example)

['377509-5',
 '2000 Rugby League World Cup',
 'Final',
 '',
 ['australia', 'position', 'new zealand'],
 [[[9, 0], 'Robbie Kearns'],
  [[12, 0], 'Scott Hill'],
  [[14, 0], 'Nathan Hindmarsh'],
  [[7, 2], 'Craig Smith'],
  [[11, 2], 'Stephen Kearney'],
  [[1, 0], 'Mat Rogers'],
  [[13, 0], 'Trent Barrett'],
  [[10, 0], 'Gorden Tallis'],
  [[5, 2], 'Henry Paul'],
  [[16, 0], 'Jason Stevens'],
  [[4, 0], 'Wendell Sailor'],
  [[17, 0], 'Chris Anderson'],
  [[15, 2], 'Nathan Cayless'],
  [[8, 2], 'Richard Swain'],
  [[9, 2], 'Quentin Pongia'],
  [[6, 2], 'Stacey Jones'],
  [[2, 2], 'Tonie Carroll'],
  [[11, 0], 'Bryan Fletcher'],
  [[0, 0], 'Darren Lockyer'],
  [[4, 2], 'Lesley Vainikolo'],
  [[0, 2], 'Richie Barnett'],
  [[15, 0], 'Darren Britt'],
  [[8, 0], 'Andrew Johns'],
  [[14, 2], 'Joe Vagana'],
  [[12, 2], 'Ruben Wiki'],
  [[7, 0], 'Shane Webcke'],
  [[5, 0], 'Brad Fittler'],
  [[13, 2], 'Robbie Paul'],
  [[10, 2], 'Matt Rua'],
  [[2, 0], 'Adam MacDougall'],
  [[16, 2], 'Logan Swann'

**changes made in the data loading script. Chnages made to include the model bert case uncased**

In [ ]:

from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import Sampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from base import BaseDataLoader
import pickle
import numpy as np
import json
import os
from tqdm import tqdm
from torch._six import string_classes
import re
import random
import copy
import itertools
import math
from multiprocessing import Pool
from functools import partial

import pdb

from model.transformers import BertTokenizer


def process_single_EL(input_data, config):
    table_id, pgTitle, secTitle, caption, headers, entities, candidate_entities, labels,_ = input_data

    cand_name = []
    cand_description = []
    cand_type = []
    for name, description, types in candidate_entities:
        if name is None:
            name=''
        if description is None:
            description=''
        if types is None:
            types=[]
        cand_name.append(config.tokenizer.encode(name, max_length=config.max_cell_length, add_special_tokens=False))
        cand_description.append(config.tokenizer.encode(description, max_length=config.max_description_length, add_special_tokens=False))
        cand_type.append([config.ent_type_vocab.get(t,0) for t in types])
    cand_name_length = [len(x) for x in cand_name]
    cand_description_length = [len(x) for x in cand_description]
    cand_type_length = [len(x) for x in cand_type]
    max_cand_name_length = max(cand_name_length)
    max_cand_description_length = max(cand_description_length)
    max_cand_type_length = max(cand_type_length)
    cand_name_padded = np.zeros([len(cand_name), max_cand_name_length], dtype=int)
    cand_description_padded = np.zeros([len(cand_description), max_cand_description_length], dtype=int)
    cand_type_padded = np.zeros([len(cand_type), max_cand_type_length], dtype=int)
    for i,(x,y,z) in enumerate(zip(cand_name_length,cand_description_length,cand_type_length)):
        cand_name_padded[i,:x] = cand_name[i]
        cand_description_padded[i,:y] = cand_description[i]
        cand_type_padded[i,:z] = cand_type[i]

    # pgEnt = config.entity_wikid2id.get(pgEnt, -1)
    pgEnt = -1

    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_headers = [config.tokenizer.encode(z, max_length=config.max_header_length, add_special_tokens=False) for z in headers]
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    tokenized_headers_length = [len(z) for z in tokenized_headers]
    input_tok += list(itertools.chain(*tokenized_headers))
    input_tok_pos += list(itertools.chain(*[list(range(z)) for z in tokenized_headers_length]))
    input_tok_type += [1]*sum(tokenized_headers_length)

    input_ent_text = []
    input_ent_type = []
    column_en_map = {}
    row_en_map = {}
    entities_index = []
    for e_i, (index, cell) in enumerate(entities):
        entities_index.append(index)
        entity_text = cell
        tokenized_ent_text = config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False)
        input_ent_text.append(tokenized_ent_text)
        input_ent_type.append(4)
        if index[1] not in column_en_map:
            column_en_map[index[1]] = [e_i]
        else:
            column_en_map[index[1]].append(e_i)
        if index[0] not in row_en_map:
            row_en_map[index[0]] = [e_i]
        else:
            row_en_map[index[0]].append(e_i)
    input_ent_length = len(input_ent_text)
    #create input mask
    tok_tok_mask = np.ones([len(input_tok), len(input_tok)], dtype=int)
    meta_ent_mask = np.ones([tokenized_meta_length, len(input_ent_text)], dtype=int)
    header_ent_mask = np.zeros([sum(tokenized_headers_length), len(input_ent_text)], dtype=int)
    start_i = 0
    header_span = {}
    for h_i, _ in enumerate(headers):
        header_span[h_i] = (start_i, start_i+tokenized_headers_length[h_i])
        start_i += tokenized_headers_length[h_i]
    for e_i, (index, _) in enumerate(entities):
        header_ent_mask[header_span[index[1]][0]:header_span[index[1]][1], e_i] = 1
    ent_header_mask = np.transpose(header_ent_mask)

    input_tok_mask = [tok_tok_mask, np.concatenate([meta_ent_mask, header_ent_mask], axis=0)]
    ent_meta_mask = np.ones([len(input_ent_text), tokenized_meta_length], dtype=int)
    
    ent_ent_mask = np.eye(len(input_ent_text), dtype=int)
    for _,e_is in column_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                ent_ent_mask[e_i_1, e_i_2] = 1
    for _,e_is in row_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                ent_ent_mask[e_i_1, e_i_2] = 1
    input_ent_mask = [np.concatenate([ent_meta_mask, ent_header_mask], axis=1), ent_ent_mask]
    # prepend pgEnt to input_ent, input_ent[0] = pgEnt
    if pgEnt!=-1:
        input_tok_mask[1] = np.concatenate([np.ones([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    else:
        input_tok_mask[1] = np.concatenate([np.zeros([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length]] + input_ent_text
    input_ent_type = [2] + input_ent_type

    new_input_ent_mask = [np.ones([len(input_ent_text), len(input_tok)], dtype=int), np.ones([len(input_ent_text), len(input_ent_text)], dtype=int)]
    new_input_ent_mask[0][1:, :] = input_ent_mask[0]
    new_input_ent_mask[1][1:, 1:] = input_ent_mask[1]
    if pgEnt==-1:
        new_input_ent_mask[1][:, 0] = 0
        new_input_ent_mask[1][0, :] = 0

    input_ent_mask = new_input_ent_mask
    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x

    return [table_id,np.array(input_tok),np.array(input_tok_type),np.array(input_tok_pos),(np.array(input_tok_mask[0]),np.array(input_tok_mask[1])),len(input_tok), \
                input_ent_text_padded,input_ent_cell_length,np.array(input_ent_type),(np.array(input_ent_mask[0]),np.array(input_ent_mask[1])),len(input_ent_text), \
                cand_name_padded,cand_name_length,cand_description_padded,cand_description_length,cand_type_padded,cand_type_length,len(cand_name_length),np.array(labels),entities_index]

class ELDataset(Dataset):

    def _preprocess(self, data_dir):
        preprocessed_filename = os.path.join(
            data_dir, "procressed_EL", self.src
        )
        preprocessed_filename += ".pickle"
        if not self.force_new and os.path.exists(preprocessed_filename):
            print("try loading preprocessed data from %s" % preprocessed_filename)
            with open(preprocessed_filename, "rb") as f:
                return pickle.load(f)
        else:
            print("try creating preprocessed data in %s" % preprocessed_filename)
            try:
                os.mkdir(os.path.join(data_dir, "procressed_EL"))
            except FileExistsError:
                pass
            with open(os.path.join(data_dir, "{}.table_entity_linking.json".format(self.src)), "r") as f:
                data = json.load(f)

        print('{} {} tables'.format(len(data),self.src))
        process_single_EL(data[0], self)
        pool = Pool(processes=10)
        processed_data = list(tqdm(pool.imap(partial(process_single_EL,config=self), data, chunksize=1000),total=len(data)))
        pool.close()
        # pdb.set_trace()

        
        with open(os.path.join(data_dir, "procressed_EL", '{}.pickle'.format(self.src)), 'wb') as f:
            pickle.dump(processed_data, f)
        # pdb.set_trace()
        return processed_data

    def __init__(self, data_dir, ent_type_vocab, max_input_tok=500, src="train", max_length = [50, 10, 10, 100], force_new=False, tokenizer = None):
        if tokenizer is not None:
            self.tokenizer = tokenizer
        else:
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.src = src
        self.force_new = force_new
        self.max_input_tok = max_input_tok
        self.max_title_length = max_length[0]
        self.max_header_length = max_length[1]
        self.max_cell_length = max_length[2]
        self.max_description_length = max_length[3]
        self.ent_type_vocab = ent_type_vocab
        self.ent_type_num = len(self.ent_type_vocab)
        self.data = self._preprocess(data_dir)
        # pdb.set_trace()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


class finetune_collate_fn_EL:
    def __init__(self, tokenizer, is_train=True):
        self.tokenizer = tokenizer
        self.is_train = is_train
    def __call__(self, raw_batch):
        batch_table_id, batch_input_tok, batch_input_tok_type, batch_input_tok_pos, batch_input_tok_mask, batch_input_tok_length, \
            batch_input_ent_text, batch_input_ent_cell_length, batch_input_ent_type, batch_input_ent_mask, batch_input_ent_length, \
            batch_cand_name,batch_cand_name_length_tmp,batch_cand_description,batch_cand_description_length_tmp,batch_cand_type,batch_cand_type_length_tmp,batch_cand_length,batch_labels,batch_entities_index = zip(*raw_batch)
        
        batch_size = len(batch_table_id)
        max_input_tok_length = max(batch_input_tok_length)
        max_input_ent_length = max(batch_input_ent_length)
        max_input_cell_length = max([z.shape[-1] for z in batch_input_ent_text])
        max_cand_name_length = max([z.shape[-1] for z in batch_cand_name])
        max_cand_description_length = max([z.shape[-1] for z in batch_cand_description])
        max_cand_type_length = max([z.shape[-1] for z in batch_cand_type])
        max_cand_size = max(batch_cand_length)

        batch_input_tok_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_type_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_pos_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_mask_padded = np.zeros([batch_size, max_input_tok_length, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_input_ent_text_padded = np.zeros([batch_size, max_input_ent_length, max_input_cell_length], dtype=int)
        batch_input_ent_text_length = np.ones([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_type_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_mask_padded = np.zeros([batch_size, max_input_ent_length, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_cand_name_padded = np.zeros([batch_size, max_cand_size, max_cand_name_length], dtype=int)
        batch_cand_name_length = np.ones([batch_size, max_cand_size], dtype=int)
        batch_cand_description_padded = np.zeros([batch_size, max_cand_size, max_cand_description_length], dtype=int)
        batch_cand_description_length = np.ones([batch_size, max_cand_size], dtype=int)
        batch_cand_type_padded = np.zeros([batch_size, max_cand_size, max_cand_type_length], dtype=int)
        batch_cand_type_length = np.ones([batch_size, max_cand_size], dtype=int)
        batch_cand_mask_padded = np.zeros([batch_size, max_cand_size], dtype=int)

        
        batch_labels_padded = np.full([batch_size, max_input_ent_length-1], -1, dtype=int)
        # pdb.set_trace()
        for i, (tok_l, ent_l, cand_l) in enumerate(zip(batch_input_tok_length, batch_input_ent_length, batch_cand_length)):
            batch_input_tok_padded[i, :tok_l] = batch_input_tok[i]
            batch_input_tok_type_padded[i, :tok_l] = batch_input_tok_type[i]
            batch_input_tok_pos_padded[i, :tok_l] = batch_input_tok_pos[i]
            batch_input_tok_mask_padded[i, :tok_l, :tok_l] = batch_input_tok_mask[i][0]
            batch_input_tok_mask_padded[i, :tok_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_tok_mask[i][1]

            batch_cand_name_padded[i, :cand_l, :batch_cand_name[i].shape[-1]] = batch_cand_name[i]
            batch_cand_name_length[i, :cand_l] = batch_cand_name_length_tmp[i]
            batch_cand_description_padded[i, :cand_l, :batch_cand_description[i].shape[-1]] = batch_cand_description[i]
            batch_cand_description_length[i, :cand_l] = batch_cand_description_length_tmp[i]
            batch_cand_type_padded[i, :cand_l, :batch_cand_type[i].shape[-1]] = batch_cand_type[i]
            batch_cand_type_length[i, :cand_l] = batch_cand_type_length_tmp[i]
            batch_cand_mask_padded[i, :cand_l] = 1

            batch_input_ent_text_padded[i, :ent_l, :batch_input_ent_text[i].shape[-1]] = batch_input_ent_text[i]
            batch_input_ent_text_length[i, :ent_l] = batch_input_ent_cell_length[i]
            batch_input_ent_type_padded[i, :ent_l] = batch_input_ent_type[i]
            batch_input_ent_mask_padded[i, :ent_l, :tok_l] = batch_input_ent_mask[i][0]
            batch_input_ent_mask_padded[i, :ent_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_ent_mask[i][1]
            batch_labels_padded[i, :ent_l-1] = batch_labels[i]
        
        batch_cand_name_length[batch_cand_name_length==0] = 1
        batch_cand_description_length[batch_cand_description_length==0] = 1
        batch_cand_type_length[batch_cand_type_length==0] = 1
                    
        batch_input_tok_padded = torch.LongTensor(batch_input_tok_padded)
        batch_input_tok_type_padded = torch.LongTensor(batch_input_tok_type_padded)
        batch_input_tok_pos_padded = torch.LongTensor(batch_input_tok_pos_padded)
        batch_input_tok_mask_padded = torch.LongTensor(batch_input_tok_mask_padded)

        batch_cand_name_padded = torch.LongTensor(batch_cand_name_padded)
        batch_cand_name_length = torch.LongTensor(batch_cand_name_length)
        batch_cand_description_padded = torch.LongTensor(batch_cand_description_padded)
        batch_cand_description_length = torch.LongTensor(batch_cand_description_length)
        batch_cand_type_padded = torch.LongTensor(batch_cand_type_padded)
        batch_cand_type_length = torch.LongTensor(batch_cand_type_length)
        batch_cand_mask_padded = torch.LongTensor(batch_cand_mask_padded)

        batch_input_ent_text_padded = torch.LongTensor(batch_input_ent_text_padded)
        batch_input_ent_text_length = torch.LongTensor(batch_input_ent_text_length)
        batch_input_ent_type_padded = torch.LongTensor(batch_input_ent_type_padded)
        batch_input_ent_mask_padded = torch.LongTensor(batch_input_ent_mask_padded)

        batch_labels_padded = torch.LongTensor(batch_labels_padded)

        return batch_table_id, batch_input_tok_padded, batch_input_tok_type_padded, batch_input_tok_pos_padded, batch_input_tok_mask_padded, \
                batch_input_ent_text_padded, batch_input_ent_text_length, batch_input_ent_type_padded, batch_input_ent_mask_padded, \
                batch_cand_name_padded, batch_cand_name_length,batch_cand_description_padded, batch_cand_description_length,batch_cand_type_padded, batch_cand_type_length, batch_cand_mask_padded, \
                batch_labels_padded,batch_entities_index

class ELLoader(DataLoader):
    """
    Base class for all data loaders
    """
    def __init__(
        self,
        dataset,
        batch_size,
        shuffle=True,
        is_train = True,
        num_workers=0,
        sampler=None,
    ):
        self.shuffle = shuffle
        if sampler is not None:
            self.shuffle = False

        self.batch_idx = 0
        self.n_samples = len(dataset)
        self.is_train = is_train
        self.collate_fn = finetune_collate_fn_EL(dataset.tokenizer, is_train=self.is_train)

        self.init_kwargs = {
            "dataset": dataset,
            "batch_size": batch_size,
            "shuffle": self.shuffle,
            "collate_fn": self.collate_fn,
            "num_workers": num_workers,
            "sampler": sampler
        }
        super().__init__(**self.init_kwargs)



In [ ]:
# load test data from [dataset].table_entity_linking.json
test_dataset = ELDataset(data_dir, type_vocab, max_input_tok=500, src="test_own", max_length = [50, 10, 10, 100], force_new=False, tokenizer = None)

100%|██████████| 231508/231508 [00:00<00:00, 916438.28B/s]


try loading preprocessed data from /content/drive/MyDrive/TURL/data/procressed_EL/test_own.pickle


In [ ]:
config.mode

0

In [ ]:
my_path='/content/drive/MyDrive/TURL/checkpoint/entity_linking/0/pytorch_model.bin'

In [ ]:
model = model_class(config, is_simple=True)
# load the checkpoint based on mode but i do it manually
#checkpoint = torch.load(f"output/EL/v2/{config.mode}/model_v1_table_0.2_0.6_0.7_10000_1e-4_candnew_0_adam/pytorch_model.bin")
checkpoint = torch.load(my_path)

model.load_state_dict(checkpoint)
model.to(device)
model.eval()

HybridTableEL(
  (table): HybridTableModel(
    (embeddings): TableHybridEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (ent_embeddings): Embedding(926135, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (type_embeddings): Embedding(5, 312)
      (ent_mask_embedding): Embedding(4, 312, padding_idx=0)
      (fusion): Linear(in_features=624, out_features=312, bias=True)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TableEncoderSimple(
      (layer): ModuleList(
        (0): TableLayerSimple(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace

In [ ]:
test_batch_size = 10
test_sampler = SequentialSampler(test_dataset)
test_dataloader = ELLoader(test_dataset, sampler=test_sampler, batch_size=test_batch_size, is_train=False)

# Eval!
print("Num examples = %d"%len(test_dataset))
print("Batch size = %d"%test_batch_size)
test_loss = 0.0
test_acc = 0.0
nb_test_steps = 0
test_results = []

for batch in tqdm(test_dataloader, desc="Evaluating"):
    table_id, input_tok, input_tok_type, input_tok_pos, input_tok_mask, \
            input_ent_text, input_ent_text_length, input_ent_type, input_ent_mask, \
            cand_name, cand_name_length,cand_description, cand_description_length,cand_type, cand_type_length, cand_mask, \
            labels,entities_index = batch
    input_tok = input_tok.to(device)
    input_tok_type = input_tok_type.to(device)
    input_tok_pos = input_tok_pos.to(device)
    input_tok_mask = input_tok_mask.to(device)
    input_ent_text = input_ent_text.to(device)
    input_ent_text_length = input_ent_text_length.to(device)
    input_ent_type = input_ent_type.to(device)
    input_ent_mask = input_ent_mask.to(device)
    cand_name = cand_name.to(device)
    cand_name_length = cand_name_length.to(device)
    cand_description = cand_description.to(device)
    cand_description_length = cand_description_length.to(device)
    cand_type = cand_type.to(device)
    cand_type_length = cand_type_length.to(device)
    cand_mask = cand_mask.to(device)
    labels = labels.to(device)
    
    if config.mode == 1:
        cand_description = None
        cand_description_length = None
    elif config.mode == 2:
        cand_type = None
        cand_type_length = None
    elif config.mode != 0:
        raise Exception
    
    with torch.no_grad():
        outputs = model(input_tok, input_tok_type, input_tok_pos, input_tok_mask,\
            input_ent_text, input_ent_text_length, input_ent_type, input_ent_mask, \
            cand_name, cand_name_length,cand_description, cand_description_length,cand_type, cand_type_length, cand_mask, \
            labels)
        loss = outputs[0]
        prediction_scores = outputs[1]
        predict_index = torch.argsort(prediction_scores.view(input_ent_text.size(0),input_ent_text.size(1)-1,-1),descending=True)
        sorted_scores = (torch.gather(prediction_scores.view(input_ent_text.size(0),input_ent_text.size(1)-1,-1),-1,predict_index)).tolist()
        predict_index = predict_index.tolist()
        acc = metric.accuracy(prediction_scores, labels.view(-1),ignore_index=-1)
        cand_length = cand_mask.sum(1).tolist()
        ent_length = (labels!=-1).sum(1).tolist()
        for i,t_id in enumerate(table_id):
            test_results.append([t_id,entities_index[i],\
                                 [x[:cand_length[i]] for x in predict_index[i][:ent_length[i]]],\
                                 [x[:cand_length[i]] for x in sorted_scores[i][:ent_length[i]]],\
                                ])
        test_loss += loss.mean().item()
        test_acc += acc.item()
    nb_test_steps += 1

test_loss = test_loss / nb_test_steps
test_acc = test_acc / nb_test_steps

result = {
    "eval_loss": test_loss,
    "eval_acc": test_acc,
}
for key in sorted(result.keys()):
    print("%s = %s"%(key, str(result[key])))

Num examples = 7291
Batch size = 10


Evaluating: 100%|██████████| 730/730 [00:49<00:00, 14.62it/s]

eval_acc = 0.8529694252226451
eval_loss = 0.6310599533458279


In [ ]:
# we dump the predictions in seperate file an use another script for official evaluation.
# The reason is that our entity linking is based on wikidata lookup. In certain cases, the candidates
# do not contain the target entity, such test example is still considered for metric calculation.
# However, since there is nothing to rank we do not pass thoses examples here. So the test examples here
# is incomplete
with open(os.path.join(data_dir,"test_own_entity_linking_results_0.pkl"),"wb") as f:
    pickle.dump(test_results, f)

<a class="anchor" id="ct"></a>
# CT
Evaluate column type annotation

In [ ]:
with open(os.path.join(data_dir, 'test.table_col_type.json'), 'r') as f:
    example = json.load(f)[0]
display(example)

['27295818-29',
 '2010–11 rangers f.c. season',
 27295818,
 'overall',
 '',
 ['competition', 'started round', 'final position / round'],
 [[[[0, 0], [26980923, 'Scottish Premier League']],
   [[1, 0], [18255941, 'UEFA Champions League']],
   [[2, 0], [20795986, 'UEFA Europa League']],
   [[3, 0], [28233480, 'League Cup']],
   [[4, 0], [28233536, 'Scottish Cup']]],
  [[[1, 1], [18255941, 'Group stage']],
   [[2, 1], [20795986, 'Round of 32']],
   [[3, 1], [28233480, '3rd Round']],
   [[4, 1], [28233536, '4th Round']]],
  [[[1, 2], [18255941, 'Group stage']],
   [[2, 2], [20795986, 'Round of 16']],
   [[3, 2], [28233480, 'Winners']],
   [[4, 2], [28233536, '5th Round']]]],
 [['time.event'], ['time.event'], ['time.event']]]

**Modifying the CT_Wiki_data_loaders.py file**

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import Sampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from base import BaseDataLoader
import pickle
import numpy as np
import json
import os
from tqdm import tqdm
from torch._six import string_classes
import re
import random
import copy
import itertools
import math
from multiprocessing import Pool
from functools import partial

import pdb

from model.transformers import BertTokenizer


def process_single_CT(input_data, config):
    table_id, pgTitle, pgEnt, secTitle, caption, headers, entities, type_annotations = input_data
    entities = [z for column in entities for z in column[:config.max_column]]
    pgEnt = config.entity_wikid2id.get(pgEnt, -1)

    tokenized_pgTitle = config.tokenizer.encode(pgTitle, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_meta = tokenized_pgTitle+\
                    config.tokenizer.encode(secTitle, max_length=config.max_title_length, add_special_tokens=False)
    if caption != secTitle:
        tokenized_meta += config.tokenizer.encode(caption, max_length=config.max_title_length, add_special_tokens=False)
    tokenized_headers = [config.tokenizer.encode(z, max_length=config.max_header_length, add_special_tokens=False) for z in headers]
    input_tok = []
    input_tok_pos = []
    input_tok_type = []
    tokenized_meta_length = len(tokenized_meta)
    input_tok += tokenized_meta
    input_tok_pos += list(range(tokenized_meta_length))
    input_tok_type += [0]*tokenized_meta_length
    tokenized_headers_length = [len(z) for z in tokenized_headers]
    input_tok += list(itertools.chain(*tokenized_headers))
    input_tok_pos += list(itertools.chain(*[list(range(z)) for z in tokenized_headers_length]))
    input_tok_type += [1]*sum(tokenized_headers_length)

    input_ent = []
    input_ent_text = []
    input_ent_type = []
    column_en_map = {}
    row_en_map = {}
    for e_i, (index, cell) in enumerate(entities):
        entity, entity_text = cell
        entity = config.entity_wikid2id.get(entity, 0)
        tokenized_ent_text = config.tokenizer.encode(entity_text, max_length=config.max_cell_length, add_special_tokens=False)
        input_ent.append(entity)
        input_ent_text.append(tokenized_ent_text)
        input_ent_type.append(4)
        if index[1] not in column_en_map:
            column_en_map[index[1]] = [e_i]
        else:
            column_en_map[index[1]].append(e_i)
        if index[0] not in row_en_map:
            row_en_map[index[0]] = [e_i]
        else:
            row_en_map[index[0]].append(e_i)
    input_ent_length = len(input_ent)
    # create column entity mask
    column_entity_mask = np.zeros([len(type_annotations), len(input_ent)], dtype=int)
    for j in range(len(type_annotations)):
        for e_i_1 in column_en_map[j]:
            column_entity_mask[j, e_i_1] = 1
    # create column header mask
    start_i = 0
    header_span = {}
    column_header_mask = np.zeros([len(type_annotations), len(input_tok)], dtype=int)
    for j in range(len(type_annotations)):
        header_span[j] = (start_i, start_i+tokenized_headers_length[j])
        column_header_mask[j, tokenized_meta_length+header_span[j][0]:tokenized_meta_length+header_span[j][1]] = 1
        start_i += tokenized_headers_length[j]
    #create input mask
    tok_tok_mask = np.ones([len(input_tok), len(input_tok)], dtype=int)
    meta_ent_mask = np.ones([tokenized_meta_length, len(input_ent)], dtype=int)
    header_ent_mask = np.zeros([sum(tokenized_headers_length), len(input_ent)], dtype=int)
    
    for e_i, (index, _) in enumerate(entities):
        header_ent_mask[header_span[index[1]][0]:header_span[index[1]][1], e_i] = 1
    ent_header_mask = np.transpose(header_ent_mask)

    input_tok_mask = [tok_tok_mask, np.concatenate([meta_ent_mask, header_ent_mask], axis=0)]
    ent_meta_mask = np.ones([len(input_ent), tokenized_meta_length], dtype=int)
    
    ent_ent_mask = np.eye(len(input_ent), dtype=int)
    for _,e_is in column_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                ent_ent_mask[e_i_1, e_i_2] = 1
    for _,e_is in row_en_map.items():
        for e_i_1 in e_is:
            for e_i_2 in e_is:
                ent_ent_mask[e_i_1, e_i_2] = 1
    input_ent_mask = [np.concatenate([ent_meta_mask, ent_header_mask], axis=1), ent_ent_mask]
    # prepend pgEnt to input_ent, input_ent[0] = pgEnt
    if pgEnt!=-1:
        input_tok_mask[1] = np.concatenate([np.ones([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    else:
        input_tok_mask[1] = np.concatenate([np.zeros([len(input_tok), 1], dtype=int),input_tok_mask[1]],axis=1)
    input_ent = [pgEnt if pgEnt!=-1 else 0] + input_ent
    input_ent_text = [tokenized_pgTitle[:config.max_cell_length]] + input_ent_text
    input_ent_type = [2] + input_ent_type

    new_input_ent_mask = [np.ones([len(input_ent), len(input_tok)], dtype=int), np.ones([len(input_ent), len(input_ent)], dtype=int)]
    new_input_ent_mask[0][1:, :] = input_ent_mask[0]
    new_input_ent_mask[1][1:, 1:] = input_ent_mask[1]
    if pgEnt==-1:
        new_input_ent_mask[1][:, 0] = 0
        new_input_ent_mask[1][0, :] = 0
    column_entity_mask = np.concatenate([np.zeros([len(type_annotations), 1], dtype=int),column_entity_mask],axis=1)

    input_ent_mask = new_input_ent_mask
    labels = np.zeros([len(type_annotations), config.type_num], dtype=int)
    for j, types in enumerate(type_annotations):
        for t in types:
            labels[j, config.type_vocab[t]] = 1
    input_ent_cell_length = [len(x) if len(x)!=0 else 1 for x in input_ent_text]
    max_cell_length = max(input_ent_cell_length)
    input_ent_text_padded = np.zeros([len(input_ent_text), max_cell_length], dtype=int)
    for i,x in enumerate(input_ent_text):
        input_ent_text_padded[i, :len(x)] = x

    return [table_id,np.array(input_tok),np.array(input_tok_type),np.array(input_tok_pos),(np.array(input_tok_mask[0]),np.array(input_tok_mask[1])),len(input_tok), \
                np.array(input_ent),input_ent_text_padded,input_ent_cell_length,np.array(input_ent_type),(np.array(input_ent_mask[0]),np.array(input_ent_mask[1])),len(input_ent), \
                column_header_mask,column_entity_mask,labels,len(labels)]

class WikiCTDataset(Dataset):

    def _preprocess(self, data_dir):
        preprocessed_filename = os.path.join(
            data_dir, "procressed_WikiCT", self.src
        )
        preprocessed_filename += ".pickle"
        if not self.force_new and os.path.exists(preprocessed_filename):
            print("try loading preprocessed data from %s" % preprocessed_filename)
            with open(preprocessed_filename, "rb") as f:
                return pickle.load(f)
        else:
            print("try creating preprocessed data in %s" % preprocessed_filename)
            try:
                os.mkdir(os.path.join(data_dir, "procressed_WikiCT"))
            except FileExistsError:
                pass
            with open(os.path.join(data_dir, "{}.table_col_type.json".format(self.src)), "r") as f:
                cols = json.load(f)

        print('{} {} tables'.format(len(cols),self.src))
        pool = Pool(processes=4)
        processed_cols = list(tqdm(pool.imap(partial(process_single_CT,config=self), cols, chunksize=1000),total=len(cols)))
        pool.close()
        # pdb.set_trace()

        
        with open(os.path.join(data_dir, "procressed_WikiCT", '{}.pickle'.format(self.src)), 'wb') as f:
            pickle.dump(processed_cols, f)
        # pdb.set_trace()
        return processed_cols

    def __init__(self, data_dir, entity_vocab, type_vocab, max_column=10, max_input_tok=500, src="train", max_length = [50, 10, 10], force_new=False, tokenizer = None):
        if tokenizer is not None:
            self.tokenizer = tokenizer
        else:

            #------change made here-----------------------------------------------------
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            #---------------------------------------------------------------
        self.src = src
        self.force_new = force_new
        self.max_input_tok = max_input_tok
        self.max_title_length = max_length[0]
        self.max_header_length = max_length[1]
        self.max_cell_length = max_length[2]
        self.max_column = max_column
        self.entity_vocab = entity_vocab
        self.entity_wikid2id = {self.entity_vocab[x]['wiki_id']:x for x in self.entity_vocab}
        self.type_vocab = type_vocab
        self.type_num = len(self.type_vocab)
        self.data = self._preprocess(data_dir)
        # pdb.set_trace()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


class finetune_collate_fn_CT:
    def __init__(self, tokenizer, is_train=True):
        self.tokenizer = tokenizer
        self.is_train = is_train
    def __call__(self, raw_batch):
        batch_table_id, batch_input_tok, batch_input_tok_type, batch_input_tok_pos, batch_input_tok_mask, batch_input_tok_length, \
            batch_input_ent, batch_input_ent_text, batch_input_ent_cell_length, batch_input_ent_type, batch_input_ent_mask, batch_input_ent_length, \
            batch_column_header_mask, batch_column_entity_mask, batch_labels, batch_col_num = zip(*raw_batch)
        
        batch_size = len(batch_table_id)
        max_input_tok_length = max(batch_input_tok_length)
        max_input_ent_length = max(batch_input_ent_length)
        max_input_cell_length = max([z.shape[-1] for z in batch_input_ent_text])
        max_input_col_num = max(batch_col_num)

        batch_input_tok_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_type_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_pos_padded = np.zeros([batch_size, max_input_tok_length], dtype=int)
        batch_input_tok_mask_padded = np.zeros([batch_size, max_input_tok_length, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_input_ent_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_text_padded = np.zeros([batch_size, max_input_ent_length, max_input_cell_length], dtype=int)
        batch_input_ent_text_length = np.ones([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_type_padded = np.zeros([batch_size, max_input_ent_length], dtype=int)
        batch_input_ent_mask_padded = np.zeros([batch_size, max_input_ent_length, max_input_tok_length+max_input_ent_length], dtype=int)

        batch_column_entity_mask_padded = np.zeros([batch_size, max_input_col_num, max_input_ent_length], dtype=int)
        batch_column_header_mask_padded = np.zeros([batch_size, max_input_col_num, max_input_tok_length], dtype=int)
        batch_labels_padded = np.zeros([batch_size, max_input_col_num, batch_labels[0].shape[-1]], dtype=int)
        batch_labels_mask = np.zeros([batch_size, max_input_col_num], dtype=int)
        
        for i, (tok_l, ent_l, col_num) in enumerate(zip(batch_input_tok_length, batch_input_ent_length, batch_col_num)):
            batch_input_tok_padded[i, :tok_l] = batch_input_tok[i]
            batch_input_tok_type_padded[i, :tok_l] = batch_input_tok_type[i]
            batch_input_tok_pos_padded[i, :tok_l] = batch_input_tok_pos[i]
            batch_input_tok_mask_padded[i, :tok_l, :tok_l] = batch_input_tok_mask[i][0]
            batch_input_tok_mask_padded[i, :tok_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_tok_mask[i][1]

            batch_input_ent_padded[i, :ent_l] = batch_input_ent[i]
            batch_input_ent_text_padded[i, :ent_l, :batch_input_ent_text[i].shape[-1]] = batch_input_ent_text[i]
            batch_input_ent_text_length[i, :ent_l] = batch_input_ent_cell_length[i]
            batch_input_ent_type_padded[i, :ent_l] = batch_input_ent_type[i]
            batch_input_ent_mask_padded[i, :ent_l, :tok_l] = batch_input_ent_mask[i][0]
            batch_input_ent_mask_padded[i, :ent_l, max_input_tok_length:max_input_tok_length+ent_l] = batch_input_ent_mask[i][1]
            batch_column_entity_mask_padded[i, :col_num, :ent_l] = batch_column_entity_mask[i]
            batch_column_entity_mask_padded[i, col_num:, 0] = 1
            batch_column_header_mask_padded[i, :col_num, :tok_l] = batch_column_header_mask[i]
            batch_column_header_mask_padded[i, col_num:, 0] = 1
            batch_labels_padded[i, :col_num] = batch_labels[i]
            batch_labels_mask[i, :col_num] = batch_labels[i].sum(1)!=0
                    
        batch_input_tok_padded = torch.LongTensor(batch_input_tok_padded)
        batch_input_tok_type_padded = torch.LongTensor(batch_input_tok_type_padded)
        batch_input_tok_pos_padded = torch.LongTensor(batch_input_tok_pos_padded)
        batch_input_tok_mask_padded = torch.LongTensor(batch_input_tok_mask_padded)

        batch_input_ent_padded = torch.LongTensor(batch_input_ent_padded)
        batch_input_ent_text_padded = torch.LongTensor(batch_input_ent_text_padded)
        batch_input_ent_text_length = torch.LongTensor(batch_input_ent_text_length)
        batch_input_ent_type_padded = torch.LongTensor(batch_input_ent_type_padded)
        batch_input_ent_mask_padded = torch.LongTensor(batch_input_ent_mask_padded)

        batch_column_entity_mask_padded = torch.FloatTensor(batch_column_entity_mask_padded)
        batch_column_header_mask_padded = torch.FloatTensor(batch_column_header_mask_padded)
        batch_labels_mask = torch.FloatTensor(batch_labels_mask)
        batch_labels_padded = torch.FloatTensor(batch_labels_padded)

        return batch_table_id, batch_input_tok_padded, batch_input_tok_type_padded, batch_input_tok_pos_padded, batch_input_tok_mask_padded, \
                batch_input_ent_text_padded, batch_input_ent_text_length, batch_input_ent_padded, batch_input_ent_type_padded, batch_input_ent_mask_padded, \
                batch_column_entity_mask_padded, batch_column_header_mask_padded, batch_labels_mask, batch_labels_padded

class CTLoader(DataLoader):
    """
    Base class for all data loaders
    """
    def __init__(
        self,
        dataset,
        batch_size,
        shuffle=True,
        is_train = True,
        num_workers=0,
        sampler=None,
    ):
        self.shuffle = shuffle
        if sampler is not None:
            self.shuffle = False

        self.batch_idx = 0
        self.n_samples = len(dataset)
        self.is_train = is_train
        self.collate_fn = finetune_collate_fn_CT(dataset.tokenizer, is_train=self.is_train)

        self.init_kwargs = {
            "dataset": dataset,
            "batch_size": batch_size,
            "shuffle": self.shuffle,
            "collate_fn": self.collate_fn,
            "num_workers": num_workers,
            "sampler": sampler
        }
        super().__init__(**self.init_kwargs)

In [ ]:
# load type vocab from type_vocab.txt
type_vocab = load_type_vocab(data_dir)
test_dataset = WikiCTDataset(data_dir, entity_vocab, type_vocab, max_input_tok=500, src="test", max_length = [50, 10, 10], force_new=False, tokenizer = None)

try loading preprocessed data from /content/drive/MyDrive/TURL/data/procressed_WikiCT/test.pickle


In [ ]:
id2type = {idx:t for t, idx in type_vocab.items()}
t2d_invalid = set()

In [ ]:
def average_precision(output, relevance_labels):
    with torch.no_grad():
        sorted_output = torch.argsort(output, dim=-1, descending=True)
        sorted_labels = torch.gather(relevance_labels, -1, sorted_output).float()
        cum_correct = torch.cumsum(sorted_labels, dim=-1)
        cum_precision = cum_correct / torch.arange(start=1,end=cum_correct.shape[-1]+1, device=cum_correct.device)[None, :]
        cum_precision = cum_precision * sorted_labels
        total_valid = torch.sum(sorted_labels, dim=-1)
        total_valid[total_valid==0] = 1
        average_precision = torch.sum(cum_precision, dim=-1)/total_valid

    return average_precision

In [ ]:
per_type_accuracy = {}
per_type_precision = {}
per_type_recall = {}
per_type_f1 = {}
map = {}
precision = {}
recall = {}
f1 = {}
per_table_result = {}

In [ ]:
'/content/drive/MyDrive/TURL/checkpoint/column_type/0/pytorch_model.bin'

'/content/drive/MyDrive/TURL/checkpoint/column_type/0/pytorch_model.bin'

In [ ]:
from tqdm.autonotebook import tqdm

#--------------------------changes made here----------------------------------------------
checkpoints = [
    "/content/drive/MyDrive/TURL/checkpoint/column_type/0/pytorch_model.bin",
    "/content/drive/MyDrive/TURL/checkpoint/column_type/1/pytorch_model.bin",
    "/content/drive/MyDrive/TURL/checkpoint/column_type/2/pytorch_model.bin",
    "/content/drive/MyDrive/TURL/checkpoint/column_type/3/pytorch_model.bin",
    "/content/drive/MyDrive/TURL/checkpoint/column_type/4/pytorch_model.bin",
    "/content/drive/MyDrive/TURL/checkpoint/column_type/5/pytorch_model.bin"

]

#------------------------------changes made here------------------------------------------------------
for mode in range(6):
    print(mode)
    config_class, model_class, _ = MODEL_CLASSES['CT']
    config = config_class.from_pretrained(config_name)
    config.class_num = len(type_vocab)
    config.mode = mode
    model = model_class(config, is_simple=True)
    checkpoint = checkpoints[mode]
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint)
    model.to(device)
    model.eval()
    eval_batch_size = 20
    eval_sampler = SequentialSampler(test_dataset)
    eval_dataloader = CTLoader(test_dataset, sampler=eval_sampler, batch_size=eval_batch_size, is_train=False)
    eval_loss = 0.0
    eval_map = 0.0
    nb_eval_steps = 0
    eval_targets = []
    eval_prediction_scores = []
    eval_pred = []
    eval_mask = []
    per_table_result[mode] = {}
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        table_ids, input_tok, input_tok_type, input_tok_pos, input_tok_mask, \
            input_ent_text, input_ent_text_length, input_ent, input_ent_type, input_ent_mask, \
            column_entity_mask, column_header_mask, labels_mask, labels = batch
        input_tok = input_tok.to(device)
        input_tok_type = input_tok_type.to(device)
        input_tok_pos = input_tok_pos.to(device)
        input_tok_mask = input_tok_mask.to(device)
        input_ent_text = input_ent_text.to(device)
        input_ent_text_length = input_ent_text_length.to(device)
        input_ent = input_ent.to(device)
        input_ent_type = input_ent_type.to(device)
        input_ent_mask = input_ent_mask.to(device)
        column_entity_mask = column_entity_mask.to(device)
        column_header_mask = column_header_mask.to(device)
        labels_mask = labels_mask.to(device)
        labels = labels.to(device)
        if mode == 1:
            input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
            input_tok = None
            input_tok_type = None
            input_tok_pos = None
            input_tok_mask = None
        elif mode == 2:
            input_tok_mask = input_tok_mask[:,:,:input_tok_mask.shape[1]]
            input_ent_text = None
            input_ent_text_length = None
            input_ent = None
            input_ent_type = None
            input_ent_mask = None
        elif mode == 3:
            input_ent = None
        elif mode == 4:
            input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
            input_tok = None
            input_tok_type = None
            input_tok_pos = None
            input_tok_mask = None
            input_ent = None
        elif mode == 5:
            input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
            input_tok = None
            input_tok_type = None
            input_tok_pos = None
            input_tok_mask = None
            input_ent_text = None
            input_ent_text_length = None
        with torch.no_grad():
            outputs = model(input_tok, input_tok_type, input_tok_pos, input_tok_mask,\
                input_ent_text, input_ent_text_length, input_ent, input_ent_type, input_ent_mask, column_entity_mask, column_header_mask, labels_mask, labels)
            loss = outputs[0]
            prediction_scores = outputs[1]
            for l_i in t2d_invalid:
                prediction_scores[:,:,l_i] = -1000
            for idx, table_id in enumerate(table_ids):
                valid = labels_mask[idx].nonzero().max().item()+1
                if table_id not in per_table_result[mode]:
                    per_table_result[mode][table_id] = [[],labels_mask[idx,:valid],labels[idx,:valid]]
                per_table_result[mode][table_id][0].append(prediction_scores[idx,:valid])
            ap = metric.average_precision(prediction_scores.view(-1, config.class_num), labels.view((-1, config.class_num)))
            map = (ap*labels_mask.view(-1)).sum()/labels_mask.sum()
            eval_loss += loss.mean().item()
            eval_map += map.item()
            eval_targets.extend(labels.view(-1, config.class_num).tolist())
            eval_prediction_scores.extend(prediction_scores.view(-1, config.class_num).tolist())
            eval_pred.extend((torch.sigmoid(prediction_scores.view(-1, config.class_num))>0.5).tolist())
            eval_mask.extend(labels_mask.view(-1).tolist())
        nb_eval_steps += 1
    print(eval_map/nb_eval_steps)
    eval_targets = np.array(eval_targets)
    eval_prediction_scores = np.array(eval_prediction_scores)
    eval_mask = np.array(eval_mask)
    eval_prediction_ranks = np.argsort(np.argsort(-eval_prediction_scores))
    eval_pred = np.array(eval_pred)
    eval_tp = eval_mask[:,np.newaxis]*eval_pred*eval_targets
    eval_precision = np.sum(eval_tp,axis=0)/np.sum(eval_mask[:,np.newaxis]*eval_pred,axis=0)
    eval_precision = np.nan_to_num(eval_precision, 1)
    eval_recall = np.sum(eval_tp,axis=0)/np.sum(eval_mask[:,np.newaxis]*eval_targets,axis=0)
    eval_recall = np.nan_to_num(eval_recall, 1)
    eval_f1 = 2*eval_precision*eval_recall/(eval_precision+eval_recall)
    eval_f1 = np.nan_to_num(eval_f1, 0)
    per_type_instance_num = np.sum(eval_mask[:,np.newaxis]*eval_targets,axis=0)
    per_type_instance_num[per_type_instance_num==0] = 1
    per_type_correct_instance_num = np.sum(eval_mask[:,np.newaxis]*(eval_prediction_ranks<eval_targets.sum(axis=1)[:,np.newaxis])*eval_targets,axis=0)
    per_type_accuracy[mode] = per_type_correct_instance_num/per_type_instance_num
    per_type_precision[mode] = eval_precision
    per_type_recall[mode] = eval_recall
    per_type_f1[mode] = eval_f1
    precision[mode] = np.sum(eval_tp)/np.sum(eval_mask[:,np.newaxis]*eval_pred)
    recall[mode] = np.sum(eval_tp)/np.sum(eval_mask[:,np.newaxis]*eval_targets)
    f1[mode] = 2*precision[mode]*recall[mode]/(precision[mode]+recall[mode])

0


Evaluating:   0%|          | 0/239 [00:00<?, ?it/s]

0.9933874786648291


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide


1


...

0.9846508577777751
4


Evaluating:   0%|          | 0/239 [00:00<?, ?it/s]

0.9616417717734141
5


Evaluating:   0%|          | 0/239 [00:00<?, ?it/s]

0.9862419815741822


In [ ]:
total_corr = 0
total_valid = 0
errors = []
for table_id, result in per_table_result[3].items():
    prediction_scores, label_mask, label = result
    prediction_scores = torch.stack(prediction_scores, 0).mean(0)
    current_corr = 0
    for col_idx, pred in enumerate(prediction_scores.argmax(-1).tolist()):
        current_corr += label[col_idx, pred].item()
    total_valid += label_mask.sum().item()
    total_corr += current_corr
    if current_corr!=label_mask.sum().item():
        errors.append(table_id)
print(total_corr/total_valid, total_valid)

0.982725527831094 13025.0


In [ ]:
for t,i in sorted(type_vocab.items(),key=lambda z:-per_type_instance_num[z[1]]):
    print('%s %.4f %.4f %.4f %.4f %.4f  %.4f %.4f'%(t, per_type_instance_num[i], per_type_f1[0][i], per_type_f1[4][i], per_type_f1[1][i], per_type_f1[3][i], per_type_f1[2][i], per_type_f1[5][i]))
    print()

people.person 3906.0000 0.9963 0.9843 0.9932 0.9932  0.9837 0.9904

location.location 3245.0000 0.9943 0.9682 0.9911 0.9863  0.9787 0.9862

organization.organization 2122.0000 0.9657 0.8941 0.9572 0.9343  0.8974 0.9507

sports.sports_team 1701.0000 0.9886 0.9536 0.9847 0.9772  0.9659 0.9871

sports.pro_athlete 1688.0000 0.9117 0.8763 0.8977 0.9066  0.8984 0.9002

time.event 1304.0000 0.9969 0.9645 0.9923 0.9843  0.9694 0.9880

soccer.football_team 822.0000 0.9469 0.8959 0.9426 0.9309  0.9173 0.9472

location.citytown 562.0000 0.8449 0.6438 0.7860 0.7681  0.7473 0.7798

location.country 538.0000 0.9788 0.9267 0.9788 0.9542  0.9292 0.9797

government.politician 495.0000 0.9261 0.8797 0.9193 0.9080  0.8750 0.9043

sports.sports_league 491.0000 0.9716 0.9457 0.9672 0.9587  0.9265 0.9724

location.administrative_division 317.0000 0.8537 0.6678 0.8213 0.8098  0.7051 0.8097

sports.sports_league_season 312.0000 0.9062 0.7783 0.8857 0.8527  0.8217 0.9011

government.u_s_congressperson 308.0000

In [ ]:
f1

{0: 0.9475334987593053,
 1: 0.9376614534313235,
 2: 0.9024452821251653,
 3: 0.926955935697437,
 4: 0.8885750030267566,
 5: 0.9332609657057854}

In [ ]:
f1

{0: 0.9475334987593053,
 1: 0.9376614534313235,
 2: 0.9024452821251653,
 3: 0.926955935697437,
 4: 0.8885750030267566,
 5: 0.9332609657057854}

In [ ]:
precision

{0: 0.949512631788343,
 1: 0.9479693889946147,
 2: 0.8990915883440167,
 3: 0.9275258836129954,
 4: 0.9054198536063821,
 5: 0.9472374112462254}

In [ ]:
recall

{0: 0.9455625990491283,
 1: 0.9275752773375594,
 2: 0.905824088748019,
 3: 0.9263866877971474,
 4: 0.8723454833597465,
 5: 0.9196909667194929}

Type mapping is used to map the types used in some other datasets to our types, so we can directly evaluate without retraining our model

In [ ]:
t2d_type_mapping = {
    'Election': ['government.election'],
    'Film': ['film.film'],
    'mountain': ['geography.mountain'],
    'Building': ['architecture.building'],
    'RadioStation': ['broadcast.radio_station'],
    'TelevisionShow': ['tv.tv_program'],
    'Country': ['location.country'],
    'Airport': ['aviation.airport'],
    'AdministrativeRegion': ['location.region'],
    'University': ['education.university'],
    'Newspaper': ['book.newspaper'],
    'FictionalCharacter': ['fictional_universe.fictional_character'],
    'Currency': ['finance.currency'],
    'Novel': ['book.book'],
    'Wrestler': ['sports.pro_athlete'],
    'swimmer': ['sports.pro_athlete'],
    'GolfPlayer': ['sports.golfer', 'sports.pro_athlete'],
    'Book': ['book.book'],
    'Political Party': ['government.political_party'],
    'Person': ['people.person'],
    'VideoGame': ['cvg.computer_videogame'],
    'Animal': ['biology.animal'],
    'PoliticalParty': ['government.political_party'],
    'BaseballPlayer': ['sports.pro_athlete'],
    'Monarch': ['royalty.monarch'],
    'Mountain': ['geography.mountain'],
    'City': ['location.citytown'],
    'Company': ['business.consumer_company'],
    'cricketer': ['sports.pro_athlete'],
    'Airline': ['aviation.airline']
}
t2d_types = set([y for _,x in t2d_type_mapping.items() for y in x])
t2d_invalid = []
for t,i in type_vocab.items():
    if t not in t2d_types:
        t2d_invalid.append(i)

In [ ]:
t2d_type_mapping = {
    'City': ['location.citytown'],
    'VideoGame': ['cvg.computer_videogame'],
    'Mountain': ['geography.mountain'],
    'Museum': [],
    'Writer': ['film.writer', 'tv.tv_writer', 'music.writer', 'book.author'],
    'Lake': [], 
    'AdministrativeRegion': ['location.administrative_division'],
    'Book': ['book.book'],
    'Saint': [],
    'Monarch': ['royalty.monarch'],
    'Bird': [],
    'Plant': [],
    'Mayor': [],
    'Currency': ['finance.currency'],
    'MovieDirector': ['film.director'],
    'Company': ['film.production_company', 'automotive.company', 'business.consumer_company', 'business.defunct_company', ],
    'Genre': ['cvg.cvg_genre', 'film.film_genre', 'broadcast.genre', 'media_common.media_genre', 'tv.tv_genre', 'music.genre'],
    'GovernmentType': ['government.governmental_body'],
    'Hospital': [],
    'Building': ['architecture.building'],
    'PoliticalParty': ['government.political_party'],
    'Language': ['language.human_language'],
    'Country': ['location.country'],
    'University': ['education.university'],
    'SportsTeam': ['sports.sports_team'],
    'RadioStation': ['broadcast.radio_station'],
    'Airport': ['aviation.airport'],
    'Airline': ['aviation.airline'],
    'Wrestler': [],
    'Newspaper': ['book.newspaper'],
    'Mammal': [],
    'MountainRange': [],
    'BaseballPlayer': ['baseball.baseball_player'],
    'AcademicJournal': [],
    'Scientist': [],
    'Continent': [],
    'Film': ['film.film']
}

t2d_types = set([y for _,x in t2d_type_mapping.items() for y in x])
t2d_invalid = []
for t,i in type_vocab.items():
    if t not in t2d_types:
        t2d_invalid.append(i)

In [ ]:
t2d_type_mapping = {
    'Film': ['film.film'],
    'Lake': [],
    'Language': ['language.human_language'],
    'Country': ['location.country'],
    'Company': ['film.production_company', 'automotive.company', 'business.consumer_company', 'business.defunct_company'],
    'Person': ['people.person'],
    'VideoGame': ['cvg.computer_videogame'],
    'City': ['location.citytown'],
    'Currency': ['finance.currency'],
    'Bird': [],
    'Mountain': ['geography.mountain'],
    'Scientist': [],
    'Plant': [],
    'TelevisionShow': ['tv.tv_program'],
    'Animal': [],
    'AdministrativeRegion': ['location.administrative_division'],
    'Genre': ['cvg.cvg_genre', 'film.film_genre', 'broadcast.genre', 'media_common.media_genre', 'tv.tv_genre', 'music.genre'],
    'Newspaper': ['book.newspaper'],
    'Airport': ['aviation.airport'],
    'AcademicJournal': [],
    'PopulatedPlace': [],
    'Wrestler': [],
    'PoliticalParty': ['government.political_party'],
    'Cricketer': ['cricket.cricket_player'],
    'Eukaryote': [],
    'Saint': [],
    'Writer': ['film.writer', 'tv.tv_writer', 'music.writer', 'book.author'],
    'Museum': [],
    'BaseballPlayer': ['baseball.baseball_player'],
    'EducationalInstitution': ['education.educational_institution'],
    'GovernmentType': ['government.governmental_body'],
    'SportsTeam': ['sports.sports_team'],   
}

reverse_type_mapping = {t2:t1 for t1,t2s in t2d_type_mapping.items() for t2 in t2s}

t2d_types = set([y for _,x in t2d_type_mapping.items() for y in x])
t2d_invalid = []
for t,i in type_vocab.items():
    if t not in t2d_types:
        t2d_invalid.append(i)

In [ ]:
errors

['56602-2',
 '29006961-1',
 '13596949-1',
 '34411-2',
 '24237524-6',
 '309890-10',
 '9634320-2',
 '27369580-2',
 '58074-2',
 '387618-1',
 '88184-95',
 '27802631-12',
 '79397-3',
 '35306913-6',
 '14688681-2',
 '23756649-2',
 '8238237-1',
 '8246459-3',
 '63065-3',
 '2754197-1',
 '28602041-3',
 '30167892-2',
 '13926824-2',
 '30889011-1',
 '32167235-1',
 '11730138-2',
 '31547882-14',
 '25391150-5',
 '335810-1',
 '33599465-1',
 '9610140-1',
 '16786378-1',
 '40554028-1',
 '944336-1',
 '160843-1',
 '343206-1',
 '235195-1',
 '1073488-1',
 '10577579-2',
 '38978487-1',
 '1432070-1',
 '2115847-1',
 '18956862-1',
 '19753692-3',
 '39956817-2',
 '2709605-1',
 '18162085-5',
 '31503340-4',
 '26248738-2',
 '26250392-12',
 '5603032-2',
 '13110459-1',
 '28873880-1',
 '10613742-2',
 '34368822-1',
 '27987339-1',
 '34736917-1',
 '34753869-1',
 '198039-4',
 '21204469-1',
 '21211-2',
 '5400266-2',
 '24598502-5',
 '9481737-2',
 '33638399-6',
 '36923967-1',
 '9272554-1',
 '1361887-2',
 '18113377-2',
 '8209610-1

In [ ]:
type_vocab

{'american_football.football_coach': 85,
 'american_football.football_conference': 15,
 'american_football.football_player': 201,
 'american_football.football_team': 253,
 'amusement_parks.park': 166,
 'amusement_parks.ride': 48,
 'architecture.architectural_structure_owner': 123,
 'architecture.building': 132,
 'architecture.structure': 68,
 'architecture.venue': 121,
 'astronomy.asteroid': 64,
 'astronomy.astronomical_discovery': 246,
 'astronomy.celestial_object': 39,
 'astronomy.constellation': 125,
 'astronomy.orbital_relationship': 209,
 'astronomy.star_system_body': 172,
 'automotive.company': 150,
 'automotive.model': 197,
 'aviation.aircraft_model': 216,
 'aviation.aircraft_owner': 170,
 'aviation.airline': 143,
 'aviation.airport': 56,
 'award.award': 138,
 'award.award_category': 14,
 'award.award_ceremony': 119,
 'award.award_discipline': 188,
 'award.award_presenting_organization': 99,
 'award.competition': 75,
 'award.hall_of_fame_inductee': 236,
 'award.recurring_competi

In [ ]:
#-----------------------Trying to do the same for the dataset as suggested by Jixiong--------------------------------

In [ ]:
# We are done with replicating the original results. Now what is left with is experimenting with new datasets

In [ ]:
# We start with Semtab

**EXPERIMENTATION WITH ANOTHER DATASET(IN THIS CASE WITH SEMTAB)**

# CT - Semtab

In [ ]:
pwd

'/content/drive/MyDrive/TURL'

In [ ]:
data_dir = '/content/drive/MyDrive/TURL/data'
type_vocab = load_type_vocab(data_dir)
#------------------------------------------------------------Same methdology as befoore
test_dataset = WikiCTDataset(data_dir, entity_vocab, type_vocab, max_input_tok=500, src="test", max_length = [50, 10, 10], force_new=False, tokenizer = None)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------

try loading preprocessed data from /content/drive/MyDrive/TURL/data/procressed_WikiCT/test.pickle


In [ ]:
id2type = {y:x for x,y in type_vocab.items()}

In [ ]:
def average_precision(output, relevance_labels):
    with torch.no_grad():
        sorted_output = torch.argsort(output, dim=-1, descending=True)
        sorted_labels = torch.gather(relevance_labels, -1, sorted_output).float()
        cum_correct = torch.cumsum(sorted_labels, dim=-1)
        cum_precision = cum_correct / torch.arange(start=1,end=cum_correct.shape[-1]+1, device=cum_correct.device)[None, :]
        cum_precision = cum_precision * sorted_labels
        total_valid = torch.sum(sorted_labels, dim=-1)
        total_valid[total_valid==0] = 1
        average_precision = torch.sum(cum_precision, dim=-1)/total_valid

    return average_precision

In [ ]:
per_type_accuracy = {}
per_type_precision = {}
per_type_recall = {}
per_type_f1 = {}
map = {}
precision = {}
recall = {}
f1 = {}

In [ ]:
'/content/drive/MyDrive/TURL/checkpoint/column_type/4/pytorch_model.bin'

'/content/drive/MyDrive/TURL/checkpoint/column_type/4/pytorch_model.bin'

In [ ]:
from tqdm.autonotebook import tqdm
checkpoint = "/content/drive/MyDrive/TURL/checkpoint/column_type/4/pytorch_model.bin"
mode = 4
print(mode)
config_class, model_class, _ = MODEL_CLASSES['CT']
config = config_class.from_pretrained(config_name)
config.class_num = len(type_vocab)
config.mode = mode
model = model_class(config, is_simple=True)
checkpoint = torch.load(checkpoint)
model.load_state_dict(checkpoint)
model.to(device)
model.eval()
eval_batch_size = 20
eval_sampler = SequentialSampler(test_dataset)
eval_dataloader = CTLoader(test_dataset, sampler=eval_sampler, batch_size=eval_batch_size, is_train=False)
eval_loss = 0.0
eval_map = 0.0
nb_eval_steps = 0
eval_targets = []
eval_prediction_scores = []
eval_pred = []
eval_mask = []
per_table_result = {}
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    table_ids, input_tok, input_tok_type, input_tok_pos, input_tok_mask, \
        input_ent_text, input_ent_text_length, input_ent, input_ent_type, input_ent_mask, \
        column_entity_mask, column_header_mask, labels_mask, labels = batch
    input_tok = input_tok.to(device)
    input_tok_type = input_tok_type.to(device)
    input_tok_pos = input_tok_pos.to(device)
    input_tok_mask = input_tok_mask.to(device)
    input_ent_text = input_ent_text.to(device)
    input_ent_text_length = input_ent_text_length.to(device)
    input_ent = input_ent.to(device)
    input_ent_type = input_ent_type.to(device)
    input_ent_mask = input_ent_mask.to(device)
    column_entity_mask = column_entity_mask.to(device)
    column_header_mask = column_header_mask.to(device)
    labels_mask = labels_mask.to(device)
    labels = labels.to(device)
    if mode == 1:
        input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
        input_tok = None
        input_tok_type = None
        input_tok_pos = None
        input_tok_mask = None
    elif mode == 2:
        input_tok_mask = input_tok_mask[:,:,:input_tok_mask.shape[1]]
        input_ent_text = None
        input_ent_text_length = None
        input_ent = None
        input_ent_type = None
        input_ent_mask = None
    elif mode == 3:
        input_ent = None
    elif mode == 4:
        input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
        input_tok = None
        input_tok_type = None
        input_tok_pos = None
        input_tok_mask = None
        input_ent = None
    elif mode == 5:
        input_ent_mask = input_ent_mask[:,:,input_tok_mask.shape[1]:]
        input_tok = None
        input_tok_type = None
        input_tok_pos = None
        input_tok_mask = None
        input_ent_text = None
        input_ent_text_length = None
    with torch.no_grad():
        outputs = model(input_tok, input_tok_type, input_tok_pos, input_tok_mask,\
            input_ent_text, input_ent_text_length, input_ent, input_ent_type, input_ent_mask, column_entity_mask, column_header_mask, labels_mask, labels)
        loss = outputs[0]
        prediction_scores = outputs[1]
        for idx, table_id in enumerate(table_ids):
            valid = labels_mask[idx].nonzero().max().item()+1
            if table_id not in per_table_result:
                per_table_result[table_id] = [[],labels_mask[idx,:valid],labels[idx,:valid]]
            per_table_result[table_id][0].append(prediction_scores[idx,:valid])
        
        eval_loss += loss.mean().item()
        eval_targets.extend(labels.view(-1, config.class_num).tolist())
        eval_prediction_scores.extend(prediction_scores.view(-1, config.class_num).tolist())
        eval_pred.extend((prediction_scores.view(-1, config.class_num)==prediction_scores.view(-1, config.class_num).max(-1)[0][:,None]).tolist())
        eval_mask.extend(labels_mask.view(-1).tolist())
    nb_eval_steps += 1
eval_targets = np.array(eval_targets)
eval_prediction_scores = np.array(eval_prediction_scores)
eval_mask = np.array(eval_mask)
eval_prediction_ranks = np.argsort(np.argsort(-eval_prediction_scores))
eval_pred = np.array(eval_pred)
eval_tp = eval_mask[:,np.newaxis]*eval_pred*eval_targets
eval_precision = np.sum(eval_tp,axis=0)/np.sum(eval_mask[:,np.newaxis]*eval_pred,axis=0)
eval_precision = np.nan_to_num(eval_precision, 1)
eval_recall = np.sum(eval_tp,axis=0)/np.sum(eval_mask[:,np.newaxis]*eval_targets,axis=0)
eval_recall = np.nan_to_num(eval_recall, 1)
eval_f1 = 2*eval_precision*eval_recall/(eval_precision+eval_recall)
eval_f1 = np.nan_to_num(eval_f1, 0)
per_type_instance_num = np.sum(eval_mask[:,np.newaxis]*eval_targets,axis=0)
per_type_instance_num[per_type_instance_num==0] = 1
per_type_correct_instance_num = np.sum(eval_mask[:,np.newaxis]*(eval_prediction_ranks<eval_targets.sum(axis=1)[:,np.newaxis])*eval_targets,axis=0)
per_type_accuracy[mode] = per_type_correct_instance_num/per_type_instance_num
per_type_precision[mode] = eval_precision
per_type_recall[mode] = eval_recall
per_type_f1[mode] = eval_f1
precision[mode] = np.sum(eval_tp)/np.sum(eval_mask[:,np.newaxis]*eval_pred)
recall[mode] = np.sum(eval_tp)/np.sum(eval_mask[:,np.newaxis]*eval_targets)
f1[mode] = 2*precision[mode]*recall[mode]/(precision[mode]+recall[mode])

4


Evaluating:   0%|          | 0/239 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
wiki_types = ['City', 'VideoGame', 'Mountain', 'Writer', 'Lake', 'AdministrativeRegion', 'Book', 'Saint', 'Monarch', 'Bird', 'Plant', 'Currency', 'Company', 'Genre', 'Building', 'PoliticalParty', 'Language', 'Country', 'University', 'SportsTeam', 'RadioStation', 'Airport', 'Wrestler', 'Newspaper', 'Mammal', 'Mayor', 'AcademicJournal', 'Scientist', 'Continent', 'Film', 'BaseballPlayer']
non_wiki_types = [x for x in type_vocab if x not in wiki_types]

In [ ]:
type_vocab

{'american_football.football_coach': 85,
 'american_football.football_conference': 15,
 'american_football.football_player': 201,
 'american_football.football_team': 253,
 'amusement_parks.park': 166,
 'amusement_parks.ride': 48,
 'architecture.architectural_structure_owner': 123,
 'architecture.building': 132,
 'architecture.structure': 68,
 'architecture.venue': 121,
 'astronomy.asteroid': 64,
 'astronomy.astronomical_discovery': 246,
 'astronomy.celestial_object': 39,
 'astronomy.constellation': 125,
 'astronomy.orbital_relationship': 209,
 'astronomy.star_system_body': 172,
 'automotive.company': 150,
 'automotive.model': 197,
 'aviation.aircraft_model': 216,
 'aviation.aircraft_owner': 170,
 'aviation.airline': 143,
 'aviation.airport': 56,
 'award.award': 138,
 'award.award_category': 14,
 'award.award_ceremony': 119,
 'award.award_discipline': 188,
 'award.award_presenting_organization': 99,
 'award.competition': 75,
 'award.hall_of_fame_inductee': 236,
 'award.recurring_competi

In [ ]:
for t,i in type_vocab.items():
    print(t, per_type_f1[4][i],per_type_instance_num[i])

royalty.noble_person 0.025974025974025976 75.0
business.business_operation 0.024793388429752067 237.0
protected_sites.listed_site 0.0 13.0
music.writer 0.0 3.0
people.ethnicity 0.8 6.0
government.government_office_or_title 0.6666666666666665 10.0
organization.non_profit_organization 0.0 1.0
business.brand 0.0 3.0
tennis.tennis_tournament 0.33333333333333337 5.0
cvg.cvg_genre 0.16666666666666669 10.0
ice_hockey.hockey_position 0.6938775510204082 32.0
sports.sports_team 0.8905109489051095 1701.0
computer.computer 0.0 3.0
metropolitan_transit.transit_line 0.5 15.0
award.award_category 0.9508196721311476 30.0
american_football.football_conference 0.0 5.0
sports.professional_sports_team 0.05839416058394161 132.0
soccer.football_world_cup 0.0 2.0
tv.tv_actor 0.014285714285714287 139.0
business.industry 0.4 4.0
music.composition 0.8842105263157894 52.0
people.person 0.9796862946772948 3906.0
broadcast.tv_channel 0.0 1.0
cricket.cricket_player 0.05128205128205127 38.0
internet.website 0.0 1.0


In [ ]:
non_wiki_types

['royalty.noble_person',
 'business.business_operation',
 'protected_sites.listed_site',
 'music.writer',
 'people.ethnicity',
 'government.government_office_or_title',
 'organization.non_profit_organization',
 'business.brand',
 'tennis.tennis_tournament',
 'cvg.cvg_genre',
 'ice_hockey.hockey_position',
 'sports.sports_team',
 'computer.computer',
 'metropolitan_transit.transit_line',
 'award.award_category',
 'american_football.football_conference',
 'sports.professional_sports_team',
 'soccer.football_world_cup',
 'tv.tv_actor',
 'business.industry',
 'music.composition',
 'people.person',
 'broadcast.tv_channel',
 'cricket.cricket_player',
 'internet.website',
 'tennis.tennis_player',
 'music.media_format',
 'tv.tv_personality',
 'film.actor',
 'film.film_genre',
 'cvg.cvg_developer',
 'business.job_title',
 'chess.chess_player',
 'tv.tv_writer',
 'broadcast.broadcast',
 'soccer.fifa',
 'cvg.cvg_publisher',
 'film.writer',
 'medicine.anatomical_structure',
 'astronomy.celestial_ob

In [ ]:
type_vocab

{'american_football.football_coach': 85,
 'american_football.football_conference': 15,
 'american_football.football_player': 201,
 'american_football.football_team': 253,
 'amusement_parks.park': 166,
 'amusement_parks.ride': 48,
 'architecture.architectural_structure_owner': 123,
 'architecture.building': 132,
 'architecture.structure': 68,
 'architecture.venue': 121,
 'astronomy.asteroid': 64,
 'astronomy.astronomical_discovery': 246,
 'astronomy.celestial_object': 39,
 'astronomy.constellation': 125,
 'astronomy.orbital_relationship': 209,
 'astronomy.star_system_body': 172,
 'automotive.company': 150,
 'automotive.model': 197,
 'aviation.aircraft_model': 216,
 'aviation.aircraft_owner': 170,
 'aviation.airline': 143,
 'aviation.airport': 56,
 'award.award': 138,
 'award.award_category': 14,
 'award.award_ceremony': 119,
 'award.award_discipline': 188,
 'award.award_presenting_organization': 99,
 'award.competition': 75,
 'award.hall_of_fame_inductee': 236,
 'award.recurring_competi

In [ ]:
total_corr = 0
total_valid = 0
errors = []
for table_id, result in per_table_result.items():
    prediction_scores, label_mask, label = result
    prediction_scores = torch.stack(prediction_scores, 0).mean(0)
    prediction_scores[:,15] = torch.where(prediction_scores[:,15]>prediction_scores[:,27],prediction_scores[:,15],prediction_scores[:,27])
    #prediction_scores += wiki_type_mask[None, :]
    pred_acc = ((prediction_scores==prediction_scores.max(-1)[0][:,None])*label).sum(-1)
    total_valid += label_mask.sum().item()
    total_corr += pred_acc.sum().item()
    if pred_acc.sum().item()!=label_mask.sum().item():
        errors.append(table_id)

In [ ]:
total_corr

121.0

In [ ]:
total_valid

185.0

In [ ]:
errors

['Microsoft_Award#0',
 'List_of_Major_League_Baseball_players_from_Europe#8',
 'Norwegian_krone#2',
 '2012%E2%80%9313_Handball-Bundesliga#1',
 'Harry_Bosch#0',
 'Ruslan_Sydykov#0',
 'Laurence_Olivier_Award_for_Most_Promising_Playwright#0',
 'List_of_Capsicum_cultivars#1',
 'List_of_newspapers_in_Norway#0',
 'Shekhar_Kapur#Future_projects#1',
 'Listed_buildings_in_Ince_Blundell#1',
 'List_of_mammals_of_California#0',
 'List_of_mammals_described_in_the_21st_century#1',
 'Chronological_list_of_saints_and_blesseds_in_the_16th_century#0',
 'Association_of_Alternative_Newsmedia#0',
 '1991%E2%80%9392_Eerste_Divisie#1',
 'Mayor_of_St._Louis#1',
 '2010%E2%80%9311_Belgian_Pro_League#6',
 'Aidan_Gillen#4',
 'Air_transportation_in_the_United_States#0',
 'Languages_of_the_United_States#1',
 'Kono_Mystery_ga_Sugoi!#16',
 '1951_Census_of_India#0',
 'List_of_lakes_of_Kazakhstan#0',
 'Languages_of_South_Asia#2',
 'List_of_Russian-language_playwrights#9',
 'Brossard#0',
 'List_of_mammals_of_Pennsylvania

In [ ]:
 for inspect_id in errors:
    print(inspect_id)
    prediction_scores, label_mask, label = per_table_result[inspect_id]
    prediction_scores = torch.stack(prediction_scores, 0).mean(0)
    for col_id in range(label.shape[0]):
        if label[col_id].sum().item()==0:
            continue
        display(id2type[label[col_id].nonzero().item()])
        display([[id2type[l],prediction_scores[col_id,l].item()] for l in prediction_scores[col_id].argsort().tolist()[::-1][:3]])

Microsoft_Award#0


'Scientist'

[['Film', 3.7781550884246826],
 ['VideoGame', 2.9057600498199463],
 ['MovieDirector', 2.8163394927978516]]

List_of_Major_League_Baseball_players_from_Europe#8


'BaseballPlayer'

...

'Film'

[['VideoGame', 8.95494270324707],
 ['Film', 3.858873128890991],
 ['Building', 3.560220241546631]]

Dead_Sea_Scrolls#10


'Book'

[['Monarch', 6.848552227020264],
 ['Mammal', 3.9416849613189697],
 ['Plant', 3.248840570449829]]

In [ ]:
label[col_id]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [ ]:
per_table_result['Baseball_Hall_of_Fame_balloting,_2015#0'][0][0].argsort()

tensor([[22,  6, 21, 24, 17, 23, 31, 20, 35, 18, 13,  0, 27, 26, 33, 16,  7,  9,
          5, 32, 25, 30, 19, 11,  8, 15,  3,  2, 29, 36, 10,  4, 14, 28, 12, 34,
          1]], device='cuda:0')